
<h1 id="Marketing-Campaign-Optimization">Marketing Campaign Optimization<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Marketing-Campaign-Optimization">¶</a></h1>



<h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-and-Prerequisites">¶</a></h2><p>This Jupyter Notebook  describes a marketing campaign optimization problem that is common in 
the banking and financial services industry. 
The problem is formulated using the Gurobi Python API and solved using the Gurobi Optimizer. 
We assume that key parameters of the mathematical optimization model of the marketing campaign problem are estimated using machine learning predictive response models. 
The marketing campaign optimization problem entails determining which products to offer to each customer 
in order 
to maximize total expected profit while satisfying various business constraints.</p>
<p>This modeling example is at the beginner level, where we assume that you know Python and that you have some knowledge about building mathematical optimization models. The reader should also consult the  <a href="https://www.gurobi.com/resources/?category-filter=documentation">documentation</a>
of the Gurobi Python API.</p>
<p><strong>Download the Repository</strong> <br/>
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/>
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-FSI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_marketing-campaign-optimization_COM_EVAL_GITHUB_&amp;utm_term=marketing-campaign-optimization&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, 
or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-FSI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_mkt-campaign-optimization_ACADEMIC_EVAL_GITHUB_&amp;utm_term=marketing-campaig-optimization&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>



<h2 id="Motivation">Motivation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Motivation">¶</a></h2><p>The main goal of marketing in the banking and financial services industry is to offer "the right product to the right customer at the right time". However, 
actually being able to achieve this goal is a complicated and challenging undertaking. What makes this particularly difficult is that companies have multiple products and operate under a complex set of business constraints. Choosing which products to offer to which customers in order to maximize the marketing return on investment and satisfy the business constraints is enormously complex.</p>
<p>Consider a major bank  that has made a deliberate effort to become a customer-focused institution, as opposed to a vertical product driven company. The goal of the bank is "to be the best at helping customers become financially better off by providing relevant solutions to their unique needs". A direct consequence of this goal is that marketing campaigns are multiple product campaigns as opposed to single product campaigns. 
This transforms the data science and campaign targeting process from a fairly simple application of individual response models into a significantly more complex process that invloves  choosing which product to offer to which customer and through which channel.</p>
<p>The marketing team of the bank are used to applying business rules to target customers directly. 
For example, 
they target 
customers solely on their product gaps or on marketers' business intuition. 
The bank's
marketers have also applied RFM type analysis where general recency, frequency, and monetary measurements as well as product gaps are used to target customers for specific offers.</p>
<p>The marketing team's current approach, which 
is widely used, 
relies on predictive response models to target customers for offers. These models estimate the probability that a customer will respond to a specific offer and can significantly increase the response rate to a product offering. However, simply knowing a customer's probability of responding to a particular offer is not enough when a company has several products to promote and other business constraints to consider in its marketing planning.</p>
<p>Generally speaking, marketing teams also face the problem of knowing which product to offer to a customer, not just which customer to offer a product. In practice, many ad hoc rules are used:</p>
<ul>
<li><p>prioritization rules based on response rates or estimated expected profitability measures</p>
</li>
<li><p>business rules to prioritize products that can be marketed</p>
</li>
<li><p>product response models to select customers for a particular campaign.</p>
</li>
</ul>
<p>One approach that is easily implemented but may not produce optimal customer contact plans relies on a measure of expected offer profitability to choose which products to offer customers. However, a shortcoming of this approach is its inability to effectively handle complex constraints on the customer contact plan.</p>
<p>To address this marketing campaign optimization problem, M. D. Cohen [1] proposed a MIP approach with data from Scotiabank. The marketing campaign optimization problem considered eleven unique offers: five investment, three lending, and three day-to-day banking offers. The investment offers included Guaranteed Investment Certificates (GICs), mutual funds, Registered Education Savings Program (RESP) and two unique discount brokerage offers. The lending offers included a mortgage and two credit card offers. 
The day-to-day banking offers included one of two Scotia online banking service offers and a deposit account acquisition. The term campaign is used here to imply one large pro-active customer contact campaign that is comprised of eleven distinct offers; it can be thought of as eleven single product campaigns that are being offered at generally the same time to a non-overlapping set of customers. Approximately 2.5 million customers were included in the potential target market for the campaign.</p>
<p>In this Jupyter Notebook, we will use this MIP approach to address the bank’s marketing campaign optimization problem. It should be noted that this approach could be used by virtually any company across various industries to optimize their marketing campaigns, while taking into account their business constraints.</p>



<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Problem-Description">¶</a></h2><p>The bank's marketing team needs to determine what products to offer to each customer in a way that maximizes the marketing campaign return on investment while considering the following constraints:</p>
<ul>
<li>limits on funding available for the campaign.</li>
<li>restrictions on the minimum number of product offers that can be made in a campaign.</li>
<li>campaign return-on-investment hurdle rates that must be met.</li>
</ul>



<h2 id="Solution-Approach">Solution Approach<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Solution-Approach">¶</a></h2><p>Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.</p>
<p>A mathematical optimization model has five components, namely:</p>
<ul>
<li>Sets and indices.</li>
<li>Parameters.</li>
<li>Decision variables.</li>
<li>Objective function(s).</li>
<li>Constraints.</li>
</ul>
<p>We now present a MIP approach for this marketing campaign optimization problem.</p>
<p>The MIP solution approach proposed by Cohen [1] is an improvement over the 
traditional myopic
approach of picking the customers that have the largest expected value for a particular product because it
produces a globally optimal solution 
from the viewpoint of the bank and allows for the effective implementation of business constraints across customers and business units. The approach accounts for limited resources and other business constraints.</p>
<p>We assume that the estimates for customer/offer expected incremental profit, costs, and business constraints serve as inputs to the marketing campaign optimization approach. The optimization phase is independent of the construction of these inputs.</p>
<p>The MIP approach
involves 
a tactical and an operational problem. 
For the tactical problem, we aggregate customers based on the individual expected profit parameters.  The estimated individual expected profit can be determined with data science techniques such as predictive response models. The key idea is to cluster the estimated individual expected profits and then consider the cluster centroids as representative of the data for all the individual customers within a single cluster. This aggregation enables the problem to be formulated as a linear programming problem so that rather than assigning offers to individual customers, the model identifies proportions within each cluster for each product offer that maximizes the marketing campaign return on investment while considering the business constraints. Typically, the number of customers in a cluster will be in the hundreds of thousands, which is the main decision variable of the tactical problem, consequently these variables can be considered as a continuous; therefore, the linear programming approach is justified.</p>
<p>The operational problem can be formulated as a MIP model, where the estimated individual expected profits and the output of the tactical model can be used as inputs to assign <em>products offers</em> to individual customers of each cluster in such a way that the  total marketing campaign return on investment is maximized.</p>



<h2 id="Tactical-Model-Formulation">Tactical Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Tactical-Model-Formulation">¶</a></h2><h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-4" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-5" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-1" type="math/tex">k \in K</script>: Index and set of clusters.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-6" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-7"><span class="mi" id="MathJax-Span-8" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-9" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-10" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-2" type="math/tex">j \in J</script>: Index and set of products.</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-11" style="width: 1.789em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-12"><span class="msubsup" id="MathJax-Span-13"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-14" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-15"><span class="mrow" id="MathJax-Span-16"><span class="mi" id="MathJax-Span-17" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-18" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-19" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-3" type="math/tex">\pi_{k,j}</script>: Expected profit to the bank from the offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-20" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-21"><span class="mi" id="MathJax-Span-22" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-23" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-24" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-4" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-25" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-26"><span class="mi" id="MathJax-Span-27" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-28" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-29" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-5" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-30" style="width: 1.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.43em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-31"><span class="msubsup" id="MathJax-Span-32"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-33" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-34"><span class="mrow" id="MathJax-Span-35"><span class="mi" id="MathJax-Span-36" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-37" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-38" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-6" type="math/tex">\nu_{k,j}</script>: Average variable cost associated with the offer of product  <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-39" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-40"><span class="mi" id="MathJax-Span-41" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-42" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-43" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-7" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-44" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-45"><span class="mi" id="MathJax-Span-46" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-47" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-48" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-8" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-49" style="width: 1.551em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.25em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-50"><span class="msubsup" id="MathJax-Span-51"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.9em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-52" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-53"><span class="mrow" id="MathJax-Span-54"><span class="mi" id="MathJax-Span-55" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>N</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub></math></span></span><script id="MathJax-Element-9" type="math/tex">N_{k}</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-56" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-57"><span class="mi" id="MathJax-Span-58" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-59" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-60" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-10" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;Q&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-61" style="width: 1.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1001.13em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-62"><span class="msubsup" id="MathJax-Span-63"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.72em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-64" style="font-family: MathJax_Math-italic;">Q</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-65"><span class="mrow" id="MathJax-Span-66"><span class="mi" id="MathJax-Span-67" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Q</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-11" type="math/tex">Q_{j}</script>: Minimum number of offers of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-68" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-69"><span class="mi" id="MathJax-Span-70" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-71" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-72" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-12" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;R&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-73" style="width: 0.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.78em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-74"><span class="mi" id="MathJax-Span-75" style="font-family: MathJax_Math-italic;">R</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>R</mi></math></span></span><script id="MathJax-Element-13" type="math/tex">R</script>: Corporate hurdle rate. This hurdle rate is used for the ROI calculation of the marketing campaign.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-76" style="width: 0.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.78em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-77"><span class="mi" id="MathJax-Span-78" style="font-family: MathJax_Math-italic;">B</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>B</mi></math></span></span><script id="MathJax-Element-14" type="math/tex">B</script>: Marketing campaign budget.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-79" style="width: 1.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.01em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-80"><span class="mi" id="MathJax-Span-81" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi></math></span></span><script id="MathJax-Element-15" type="math/tex">M</script>: Big M penalty. This penalty is associated with corrections on the budget that are necessary to satisfy other business constraints.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-82" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-83"><span class="msubsup" id="MathJax-Span-84"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-85" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-86"><span class="mrow" id="MathJax-Span-87"><span class="mi" id="MathJax-Span-88" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-89" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-90" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-91" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-92" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-16" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-93" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-94"><span class="mi" id="MathJax-Span-95" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-96" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-97" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-17" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-98" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-99"><span class="mi" id="MathJax-Span-100" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-101" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-102" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-18" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-103" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.503em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-104"><span class="mi" id="MathJax-Span-105" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-106" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-107" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.211em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>z</mi><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-19" type="math/tex">z \geq 0</script>: Increase in budget in order to have a feasible campaign.</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-Function">¶</a></h3><ul>
<li><strong>Total profit</strong>. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-108" style="width: 100%; display: inline-block; min-width: 19.705em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 19.705em;"><span style="position: absolute; clip: rect(2.443em, 1015.54em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-109"><span class="mtable" id="MathJax-Span-110" style="min-width: 19.705em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 19.705em;"><span style="display: inline-block; position: absolute; width: 15.539em; height: 0px; clip: rect(-1.545em, 1015.54em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.795em;"><span style="position: absolute; clip: rect(2.443em, 1015.54em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 15.539em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1015.54em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.795em;"><span class="mtd" id="MathJax-Span-114"><span class="mrow" id="MathJax-Span-115"><span class="mtext" id="MathJax-Span-116" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-117" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-118" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-119" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-120" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-121" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-122"><span class="mrow" id="MathJax-Span-123"><span class="mi" id="MathJax-Span-124" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-125" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-126" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-127" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-128" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-129"><span class="mrow" id="MathJax-Span-130"><span class="mi" id="MathJax-Span-131" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-132" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-133" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-134" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-135" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-136"><span class="mrow" id="MathJax-Span-137"><span class="mi" id="MathJax-Span-138" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-139" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-140" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-141" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-142" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-143" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-144"><span class="mrow" id="MathJax-Span-145"><span class="mi" id="MathJax-Span-146" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-147" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-148" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-149" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-150" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-151" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-152" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-0"><span class="mrow" id="MathJax-Span-112"><span class="mtext" id="MathJax-Span-113" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>−</mo><mi>M</mi><mo>⋅</mo><mi>z</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-20" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Number of offers</strong>. Maximum number of offers of products for each cluster is limited by the number of customers in the cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-153" style="width: 100%; display: inline-block; min-width: 14.051em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 14.051em;"><span style="position: absolute; clip: rect(2.443em, 1009.88em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-154"><span class="mtable" id="MathJax-Span-155" style="min-width: 14.051em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 14.051em;"><span style="display: inline-block; position: absolute; width: 9.884em; height: 0px; clip: rect(-1.545em, 1009.88em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -4.938em;"><span style="position: absolute; clip: rect(2.443em, 1009.88em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.884em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1009.88em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -4.938em;"><span class="mtd" id="MathJax-Span-159"><span class="mrow" id="MathJax-Span-160"><span class="munderover" id="MathJax-Span-161"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-162" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-163"><span class="mrow" id="MathJax-Span-164"><span class="mi" id="MathJax-Span-165" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-166" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-168" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-169" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-170"><span class="mrow" id="MathJax-Span-171"><span class="mi" id="MathJax-Span-172" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-173" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-174" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-175" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-176" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.9em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-177" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-178"><span class="mrow" id="MathJax-Span-179"><span class="mi" id="MathJax-Span-180" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-181" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-182" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-183" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-184" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-185" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1"><span class="mrow" id="MathJax-Span-157"><span class="mtext" id="MathJax-Span-158" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><msub><mi>N</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-21" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}</script><ul>
<li><strong>Budget</strong>. The marketing campaign budget constraint enforces that the total cost of the campaign should be less than the budget campaign. There is the possibility of increasing the budget to ensure the feasibility of the model, the minimum number of offers for all the product may require this increase in the budget.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-186" style="width: 100%; display: inline-block; min-width: 14.884em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 14.884em;"><span style="position: absolute; clip: rect(2.443em, 1010.78em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-187"><span class="mtable" id="MathJax-Span-188" style="min-width: 14.884em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 14.884em;"><span style="display: inline-block; position: absolute; width: 10.777em; height: 0px; clip: rect(-1.545em, 1010.78em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -5.354em;"><span style="position: absolute; clip: rect(2.443em, 1010.78em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 10.777em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1010.78em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -5.354em;"><span class="mtd" id="MathJax-Span-192"><span class="mrow" id="MathJax-Span-193"><span class="munderover" id="MathJax-Span-194"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-195" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-196"><span class="mrow" id="MathJax-Span-197"><span class="mi" id="MathJax-Span-198" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-199" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-200" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-201" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-202" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-203"><span class="mrow" id="MathJax-Span-204"><span class="mi" id="MathJax-Span-205" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-206" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-207" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-208" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-209" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-210"><span class="mrow" id="MathJax-Span-211"><span class="mi" id="MathJax-Span-212" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-213" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-214" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-215" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-216" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-217" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-218"><span class="mrow" id="MathJax-Span-219"><span class="mi" id="MathJax-Span-220" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-221" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-222" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-223" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mi" id="MathJax-Span-224" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">B</span><span class="mo" id="MathJax-Span-225" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-226" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-2"><span class="mrow" id="MathJax-Span-190"><span class="mtext" id="MathJax-Span-191" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><mi>B</mi><mo>+</mo><mi>z</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-22" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}</script><ul>
<li><strong>Offers limit</strong>. Minimum number of offers of each product.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Q&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-227" style="width: 100%; display: inline-block; min-width: 13.634em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 13.634em;"><span style="position: absolute; clip: rect(2.443em, 1009.47em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-228"><span class="mtable" id="MathJax-Span-229" style="min-width: 13.634em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 13.634em;"><span style="display: inline-block; position: absolute; width: 9.467em; height: 0px; clip: rect(-1.545em, 1009.47em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -4.759em;"><span style="position: absolute; clip: rect(2.443em, 1009.47em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1009.47em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -4.759em;"><span class="mtd" id="MathJax-Span-233"><span class="mrow" id="MathJax-Span-234"><span class="munderover" id="MathJax-Span-235"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-236" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-237"><span class="mrow" id="MathJax-Span-238"><span class="mi" id="MathJax-Span-239" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-240" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-241" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-242" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-243" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-244"><span class="mrow" id="MathJax-Span-245"><span class="mi" id="MathJax-Span-246" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-247" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-248" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-249" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-250" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.72em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-251" style="font-family: MathJax_Math-italic;">Q</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-252"><span class="mrow" id="MathJax-Span-253"><span class="mi" id="MathJax-Span-254" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-255" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-256" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-257" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-258" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-259" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-3"><span class="mrow" id="MathJax-Span-231"><span class="mtext" id="MathJax-Span-232" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>Q</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-23" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}</script><ul>
<li><strong>ROI</strong>. The minimum ROI constraint ensures that the ratio of total profits over cost is at least one plus the corporate hurdle rate.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;R&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-260" style="width: 100%; display: inline-block; min-width: 23.455em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 23.455em;"><span style="position: absolute; clip: rect(2.443em, 1019.29em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-261"><span class="mtable" id="MathJax-Span-262" style="min-width: 23.455em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 23.455em;"><span style="display: inline-block; position: absolute; width: 19.289em; height: 0px; clip: rect(-1.545em, 1019.29em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -9.64em;"><span style="position: absolute; clip: rect(2.443em, 1019.29em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 19.289em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1019.29em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -9.64em;"><span class="mtd" id="MathJax-Span-266"><span class="mrow" id="MathJax-Span-267"><span class="munderover" id="MathJax-Span-268"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-269" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-270"><span class="mrow" id="MathJax-Span-271"><span class="mi" id="MathJax-Span-272" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-273" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-274" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-275" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-276" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-277"><span class="mrow" id="MathJax-Span-278"><span class="mi" id="MathJax-Span-279" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-280" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-281" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-282" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-283" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-284"><span class="mrow" id="MathJax-Span-285"><span class="mi" id="MathJax-Span-286" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-287" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-288" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-289" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-290" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-291" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-292"><span class="mrow" id="MathJax-Span-293"><span class="mi" id="MathJax-Span-294" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-295" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-296" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-297" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mo" id="MathJax-Span-298" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="mn" id="MathJax-Span-299" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-300" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-301" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">R</span><span class="mo" id="MathJax-Span-302" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-303" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-304" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-305" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-306"><span class="mrow" id="MathJax-Span-307"><span class="mi" id="MathJax-Span-308" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-309" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-310" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-311" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-312" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-313"><span class="mrow" id="MathJax-Span-314"><span class="mi" id="MathJax-Span-315" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-316" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-317" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-318" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-319" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-320"><span class="mrow" id="MathJax-Span-321"><span class="mi" id="MathJax-Span-322" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-323" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-324" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-325" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-326" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-327" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-328"><span class="mrow" id="MathJax-Span-329"><span class="mi" id="MathJax-Span-330" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-331" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-332" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-4"><span class="mrow" id="MathJax-Span-264"><span class="mtext" id="MathJax-Span-265" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mo stretchy="false">(</mo><mn>1</mn><mo>+</mo><mi>R</mi><mo stretchy="false">)</mo><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-24" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}</script>



<h2 id="Operational-Model-Formulation">Operational Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Operational-Model-Formulation">¶</a></h2><p>Once the optimal values <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-333" style="width: 1.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.43em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-334"><span class="msubsup" id="MathJax-Span-335"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-336" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-337"><span class="mrow" id="MathJax-Span-338"><span class="mi" id="MathJax-Span-339" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-340" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-341" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-25" type="math/tex">y_{k,j}</script>, for all <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-342" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-343"><span class="mi" id="MathJax-Span-344" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-345" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-346" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-26" type="math/tex">j \in J</script> and <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-347" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-348"><span class="mi" id="MathJax-Span-349" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-350" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-351" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-27" type="math/tex">k \in K</script>, of the Tactical model have been found, we should determine which individual customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-352" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-353"><span class="mi" id="MathJax-Span-354" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-28" type="math/tex">k</script> should get an offer of a product. Suppose that for a given cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-355" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-356"><span class="mi" id="MathJax-Span-357" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-358" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-359" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-29" type="math/tex">k \in K</script>, the allocation of offers of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-360" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.84em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-361"><span class="msubsup" id="MathJax-Span-362"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-363" style="font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="mn" id="MathJax-Span-364" style="font-size: 70.7%; font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>j</mi><mn>1</mn></msub></math></span></span><script id="MathJax-Element-30" type="math/tex">j_1</script> and <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-365" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.84em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-366"><span class="msubsup" id="MathJax-Span-367"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-368" style="font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="mn" id="MathJax-Span-369" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>j</mi><mn>2</mn></msub></math></span></span><script id="MathJax-Element-31" type="math/tex">j_2</script> are positive, i.e. <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;gt;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-370" style="width: 4.408em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.634em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.57em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-371"><span class="msubsup" id="MathJax-Span-372"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-373" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-374"><span class="mrow" id="MathJax-Span-375"><span class="mi" id="MathJax-Span-376" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-377" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-378"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.3em, 4.289em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-379" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.301em;"><span class="mn" id="MathJax-Span-380" style="font-size: 50%; font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-381" style="font-family: MathJax_Main; padding-left: 0.301em;">&gt;</span><span class="mn" id="MathJax-Span-382" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><msub><mi>j</mi><mn>1</mn></msub></mrow></msub><mo>&gt;</mo><mn>0</mn></math></span></span><script id="MathJax-Element-32" type="math/tex">y_{k,j_1} > 0</script> and <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;gt;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-383" style="width: 4.408em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.634em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.57em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-384"><span class="msubsup" id="MathJax-Span-385"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-386" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-387"><span class="mrow" id="MathJax-Span-388"><span class="mi" id="MathJax-Span-389" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-390" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-391"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.3em, 4.289em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-392" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.301em;"><span class="mn" id="MathJax-Span-393" style="font-size: 50%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-394" style="font-family: MathJax_Main; padding-left: 0.301em;">&gt;</span><span class="mn" id="MathJax-Span-395" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><msub><mi>j</mi><mn>2</mn></msub></mrow></msub><mo>&gt;</mo><mn>0</mn></math></span></span><script id="MathJax-Element-33" type="math/tex">y_{k,j_2} > 0</script>. Then, <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-396" style="width: 2.086em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.73em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-397"><span class="msubsup" id="MathJax-Span-398"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-399" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-400"><span class="mrow" id="MathJax-Span-401"><span class="mi" id="MathJax-Span-402" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-403" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-404"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.3em, 4.289em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-405" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.301em;"><span class="mn" id="MathJax-Span-406" style="font-size: 50%; font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.075em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><msub><mi>j</mi><mn>1</mn></msub></mrow></msub></math></span></span><script id="MathJax-Element-34" type="math/tex">y_{k,j_1}</script> and <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-407" style="width: 2.086em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.73em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-408"><span class="msubsup" id="MathJax-Span-409"><span style="display: inline-block; position: relative; width: 1.729em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-410" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-411"><span class="mrow" id="MathJax-Span-412"><span class="mi" id="MathJax-Span-413" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-414" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-415"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.3em, 4.289em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-416" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.301em;"><span class="mn" id="MathJax-Span-417" style="font-size: 50%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.075em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><msub><mi>j</mi><mn>2</mn></msub></mrow></msub></math></span></span><script id="MathJax-Element-35" type="math/tex">y_{k,j_2}</script> of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-418" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-419"><span class="mi" id="MathJax-Span-420" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-36" type="math/tex">k</script> must be offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-421" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.84em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-422"><span class="msubsup" id="MathJax-Span-423"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-424" style="font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="mn" id="MathJax-Span-425" style="font-size: 70.7%; font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>j</mi><mn>1</mn></msub></math></span></span><script id="MathJax-Element-37" type="math/tex">j_1</script> and <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-426" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.84em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-427"><span class="msubsup" id="MathJax-Span-428"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.42em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-429" style="font-family: MathJax_Math-italic;">j</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="mn" id="MathJax-Span-430" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>j</mi><mn>2</mn></msub></math></span></span><script id="MathJax-Element-38" type="math/tex">j_2</script>, respectively. The optimal way to do that is to solve an assignment problem using the estimated expected profit for the individual customers and not the one for clusters.</p>
<p>We now provide a formulation of the operational problem.</p>
<h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-431" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-432"><span class="mi" id="MathJax-Span-433" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-434" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-435" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-436" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-437"><span class="mrow" id="MathJax-Span-438"><span class="mi" id="MathJax-Span-439" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-39" type="math/tex">i \in I^{k}</script>: Index and set of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-440" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-441"><span class="mi" id="MathJax-Span-442" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-443" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-444" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-40" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-445" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-446"><span class="mi" id="MathJax-Span-447" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-448" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-449" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-450" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-451"><span class="mrow" id="MathJax-Span-452"><span class="mi" id="MathJax-Span-453" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-41" type="math/tex">j \in J^{k}</script>: Index and subset of products offered to customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-454" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-455"><span class="mi" id="MathJax-Span-456" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-457" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-458" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-42" type="math/tex">k \in K</script> , where <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mo&gt;:&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;gt;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-459" style="width: 12.027em; display: inline-block;"><span style="display: inline-block; position: relative; width: 10.003em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1009.94em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-460"><span class="msubsup" id="MathJax-Span-461"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-462" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-463"><span class="mrow" id="MathJax-Span-464"><span class="mi" id="MathJax-Span-465" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-466" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-467" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mi" id="MathJax-Span-468" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-469" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-470" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-471" style="font-family: MathJax_Main; padding-left: 0.301em;">:</span><span class="msubsup" id="MathJax-Span-472" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-473" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-474"><span class="mrow" id="MathJax-Span-475"><span class="mi" id="MathJax-Span-476" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-477" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-478" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-479" style="font-family: MathJax_Main; padding-left: 0.301em;">&gt;</span><span class="mn" id="MathJax-Span-480" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span><span class="mo" id="MathJax-Span-481" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>=</mo><mo fence="false" stretchy="false">{</mo><mi>j</mi><mo>∈</mo><mi>J</mi><mo>:</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>&gt;</mo><mn>0</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-43" type="math/tex">J^{k} = \{ j \in J: y_{k,j} > 0 \}</script> .</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-482" style="width: 2.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.85em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-483"><span class="msubsup" id="MathJax-Span-484"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-485" style="font-family: MathJax_Math-italic;">r</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-486"><span class="mrow" id="MathJax-Span-487"><span class="mi" id="MathJax-Span-488" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-489" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-490" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-491" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-492" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>r</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-44" type="math/tex">r_{k,i,j}</script>: Expected individual profit of customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-493" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-494"><span class="mi" id="MathJax-Span-495" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-496" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-497" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-498" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-499"><span class="mrow" id="MathJax-Span-500"><span class="mi" id="MathJax-Span-501" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-45" type="math/tex">i \in I^{k}</script> from  offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-502" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-503"><span class="mi" id="MathJax-Span-504" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-505" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-506" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-507" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-508"><span class="mrow" id="MathJax-Span-509"><span class="mi" id="MathJax-Span-510" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-46" type="math/tex">j \in J^{k}</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;Y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;&amp;#x230A;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo fence="false" stretchy="false"&gt;&amp;#x230B;&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-511" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-512"><span class="msubsup" id="MathJax-Span-513"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-514" style="font-family: MathJax_Math-italic;">Y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-515"><span class="mrow" id="MathJax-Span-516"><span class="mi" id="MathJax-Span-517" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-518" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-519" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-520" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-521" style="font-family: MathJax_Main; padding-left: 0.301em;">⌊</span><span class="msubsup" id="MathJax-Span-522"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-523" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-524"><span class="mrow" id="MathJax-Span-525"><span class="mi" id="MathJax-Span-526" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-527" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-528" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-529" style="font-family: MathJax_Main;">⌋</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>=</mo><mo fence="false" stretchy="false">⌊</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo fence="false" stretchy="false">⌋</mo></math></span></span><script id="MathJax-Element-47" type="math/tex">Y_{k,j} = \lfloor y_{k,j} \rfloor </script>: Number of customers in cluster k that will get an offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-530" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-531"><span class="mi" id="MathJax-Span-532" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-533" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-534" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-535" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-536"><span class="mrow" id="MathJax-Span-537"><span class="mi" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-48" type="math/tex">j \in J^{k}</script>.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-539" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-540"><span class="msubsup" id="MathJax-Span-541"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-542" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-543"><span class="mrow" id="MathJax-Span-544"><span class="mi" id="MathJax-Span-545" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-546" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-547" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-548" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-549" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-550" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-551" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-552" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-553" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-554" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-555" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-49" type="math/tex">x_{k,i,j} \in \{0,1 \}</script>: This variable is equal to 1, if product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-50-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-556" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-557"><span class="mi" id="MathJax-Span-558" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-559" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-560" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-561" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-562"><span class="mrow" id="MathJax-Span-563"><span class="mi" id="MathJax-Span-564" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-50" type="math/tex">j \in J^{k}</script>  is offered to customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-51-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-565" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-566"><span class="mi" id="MathJax-Span-567" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-568" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-569" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-570" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-571"><span class="mrow" id="MathJax-Span-572"><span class="mi" id="MathJax-Span-573" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-51" type="math/tex">i \in I^{k}</script>, and 0 otherwise.</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-Function">¶</a></h3><ul>
<li><strong>Total profit</strong>. Maximize total individual profit.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0_1"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-52-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-574" style="width: 100%; display: inline-block; min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.813em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-575"><span class="mtable" id="MathJax-Span-576" style="min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.813em;"><span style="display: inline-block; position: absolute; width: 14.646em; height: 0px; clip: rect(-1.664em, 1014.65em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -7.318em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 14.646em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1014.65em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -7.318em;"><span class="mtd" id="MathJax-Span-580"><span class="mrow" id="MathJax-Span-581"><span class="mtext" id="MathJax-Span-582" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-583" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-584" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-585" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-586" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-587" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-588"><span class="mrow" id="MathJax-Span-589"><span class="mi" id="MathJax-Span-590" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-591" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-592" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-593" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-594" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-595"><span class="mrow" id="MathJax-Span-596"><span class="mi" id="MathJax-Span-597" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-598" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-599"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-600" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-601"><span class="mrow" id="MathJax-Span-602"><span class="mi" id="MathJax-Span-603" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-604" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-605" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-606"><span class="mrow" id="MathJax-Span-607"><span class="mi" id="MathJax-Span-608" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-609" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-610"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-611" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-612"><span class="mrow" id="MathJax-Span-613"><span class="mi" id="MathJax-Span-614" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-615" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-616" style="font-family: MathJax_Math-italic;">r</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-617"><span class="mrow" id="MathJax-Span-618"><span class="mi" id="MathJax-Span-619" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-620" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-621" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-622" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-623" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-624" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-625" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-626" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-627"><span class="mrow" id="MathJax-Span-628"><span class="mi" id="MathJax-Span-629" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-630" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-631" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-632" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-633" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-0_1"><span class="mrow" id="MathJax-Span-578"><span class="mtext" id="MathJax-Span-579" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0_1"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>r</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-52" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Product offers</strong>. Allocate offers of a product to customers of each cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1_1"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-53-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-634" style="width: 100%; display: inline-block; min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.158em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-635"><span class="mtable" id="MathJax-Span-636" style="min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.158em;"><span style="display: inline-block; position: absolute; width: 13.991em; height: 0px; clip: rect(-1.604em, 1013.99em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.021em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.991em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.99em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.021em;"><span class="mtd" id="MathJax-Span-640"><span class="mrow" id="MathJax-Span-641"><span class="munderover" id="MathJax-Span-642"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-643" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-644"><span class="mrow" id="MathJax-Span-645"><span class="mi" id="MathJax-Span-646" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-647" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-648"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-649" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-650"><span class="mrow" id="MathJax-Span-651"><span class="mi" id="MathJax-Span-652" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-653" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-654" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-655"><span class="mrow" id="MathJax-Span-656"><span class="mi" id="MathJax-Span-657" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-658" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-659" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-660" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-661" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-662" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-663" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-664" style="font-family: MathJax_Math-italic;">Y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-665"><span class="mrow" id="MathJax-Span-666"><span class="mi" id="MathJax-Span-667" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-668" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-669" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-670" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-671" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-672" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-673" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-674" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-675" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="texatom" id="MathJax-Span-676"><span class="mrow" id="MathJax-Span-677"><span class="mi" id="MathJax-Span-678" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-679" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-680" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-681" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-682" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, -0.057em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1_1"><span class="mrow" id="MathJax-Span-638"><span class="mtext" id="MathJax-Span-639" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1_1"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>=</mo><msub><mi>Y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-53" type="math/tex; mode=display">\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}</script><ul>
<li><strong>Offers limit</strong>. At most one product may be offered to a customer of a cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2_1"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-54-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-683" style="width: 100%; display: inline-block; min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.086em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-684"><span class="mtable" id="MathJax-Span-685" style="min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.086em;"><span style="display: inline-block; position: absolute; width: 12.92em; height: 0px; clip: rect(-1.664em, 1012.92em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -6.485em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 12.92em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1012.92em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.485em;"><span class="mtd" id="MathJax-Span-689"><span class="mrow" id="MathJax-Span-690"><span class="munderover" id="MathJax-Span-691"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-692" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-693"><span class="mrow" id="MathJax-Span-694"><span class="mi" id="MathJax-Span-695" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-696" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-697"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-698" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-699"><span class="mrow" id="MathJax-Span-700"><span class="mi" id="MathJax-Span-701" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-702" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-703" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-704"><span class="mrow" id="MathJax-Span-705"><span class="mi" id="MathJax-Span-706" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-707" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-708" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-709" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-710" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-711" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-712" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-713" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-714" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-715" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-716" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-717" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-718" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.598em;"><span class="texatom" id="MathJax-Span-719"><span class="mrow" id="MathJax-Span-720"><span class="mi" id="MathJax-Span-721" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-722" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-723" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-724" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-725" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-2_1"><span class="mrow" id="MathJax-Span-687"><span class="mtext" id="MathJax-Span-688" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2_1"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-54" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}</script><ul>
<li><strong>Binary constraints</strong>. Either a product offer is given to a customer of cluster k or not.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3_1"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-55-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-726" style="width: 100%; display: inline-block; min-width: 20.42em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 20.42em;"><span style="position: absolute; clip: rect(2.979em, 1016.31em, 4.527em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-727"><span class="mtable" id="MathJax-Span-728" style="min-width: 20.42em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 20.42em;"><span style="display: inline-block; position: absolute; width: 16.313em; height: 0px; clip: rect(-1.009em, 1016.31em, 0.539em, -999.997em); top: 0em; left: 50%; margin-left: -8.152em;"><span style="position: absolute; clip: rect(2.979em, 1016.31em, 4.527em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 16.313em; height: 0px;"><span style="position: absolute; clip: rect(2.92em, 1016.31em, 4.467em, -999.997em); top: -3.926em; left: 50%; margin-left: -8.152em;"><span class="mtd" id="MathJax-Span-732"><span class="mrow" id="MathJax-Span-733"><span class="msubsup" id="MathJax-Span-734"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-735" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-736"><span class="mrow" id="MathJax-Span-737"><span class="mi" id="MathJax-Span-738" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-739" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-740" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-741" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-742" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-743" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-744" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-745" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-746" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-747" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-748" style="font-family: MathJax_Main;">}</span><span class="mspace" id="MathJax-Span-749" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-750" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-751" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-752" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-753" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-754" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.598em;"><span class="texatom" id="MathJax-Span-755"><span class="mrow" id="MathJax-Span-756"><span class="mi" id="MathJax-Span-757" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-758" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-759" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-760" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-761" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-762" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="texatom" id="MathJax-Span-763"><span class="mrow" id="MathJax-Span-764"><span class="mi" id="MathJax-Span-765" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-766" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-767" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-768" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-769" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-0.89em, 1001.19em, 0.479em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -3.926em; right: 0em;"><span class="mtd" id="mjx-eqn-3_1"><span class="mrow" id="MathJax-Span-730"><span class="mtext" id="MathJax-Span-731" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3_1"><mtext>(3)</mtext></mtd><mtd><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-55" type="math/tex; mode=display">\begin{equation}
x_{k,i,j} \in \{0,1 \} \quad \forall i \in I^{k},  j \in J^{k}, k \in K
\tag{3}
\end{equation}</script>



<h2 id="Problem-Instance">Problem Instance<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Problem-Instance">¶</a></h2><p>We consider two products, ten customers, and two clusters of customers. The corporate hurdle-rate is twenty percent.</p>
<h3 id="Tactical-problem-data">Tactical problem data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Tactical-problem-data">¶</a></h3><p>The following table defines the expected profit of an average customer in each cluster when offered a product.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-56-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-770" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-771"><span class="mi" id="MathJax-Span-772" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-773" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-774" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-775" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-56" type="math/tex">\$2,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-57-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-776" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-777"><span class="mi" id="MathJax-Span-778" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-779" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-780" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-781" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-57" type="math/tex">\$1,000</script></td>
</tr>
<tr>
<td>cluster 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-58-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-782" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-783"><span class="mi" id="MathJax-Span-784" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-785" style="font-family: MathJax_Main;">3</span><span class="mo" id="MathJax-Span-786" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-787" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-58" type="math/tex">\$3,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-59-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-788" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-789"><span class="mi" id="MathJax-Span-790" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-791" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-792" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-793" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-59" type="math/tex">\$2,000</script></td>
</tr>
</tbody>
</table>
<p>The expected cost of offering a product to an average customer in a cluster is determined by the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-60-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-794" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-795"><span class="mi" id="MathJax-Span-796" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-797" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-60" type="math/tex">\$200</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-61-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-798" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-799"><span class="mi" id="MathJax-Span-800" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-801" style="font-family: MathJax_Main;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>100</mn></math></span></span><script id="MathJax-Element-61" type="math/tex">\$100</script></td>
</tr>
<tr>
<td>cluster 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;300&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-62-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-802" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-803"><span class="mi" id="MathJax-Span-804" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-805" style="font-family: MathJax_Main;">300</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>300</mn></math></span></span><script id="MathJax-Element-62" type="math/tex">\$300</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-63-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-806" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-807"><span class="mi" id="MathJax-Span-808" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-809" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-63" type="math/tex">\$200</script></td>
</tr>
</tbody>
</table>
<p>The budget available for the marketing campaign is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-64-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-810" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-811"><span class="mi" id="MathJax-Span-812" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-813" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-64" type="math/tex">\$200</script>.</p>
<p>The number of customers in each cluster is given by the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Num. Customers</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td>5</td>
</tr>
<tr>
<td>cluster 2</td>
<td>5</td>
</tr>
</tbody>
</table>
<p>The minimum number of offers of each product is provided in the following table,</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Min Offers</th>
</tr>
</thead>
<tbody>
<tr>
<td>product 1</td>
<td>2</td>
</tr>
<tr>
<td>product 2</td>
<td>2</td>
</tr>
</tbody>
</table>
<h3 id="Operational-problem-data">Operational problem data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Operational-problem-data">¶</a></h3><p>The following table shows the expected profit of each customer in each cluster when offered a product.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1, customer 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;050&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-65-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-814" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-815"><span class="mi" id="MathJax-Span-816" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-817" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-818" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-819" style="font-family: MathJax_Main; padding-left: 0.142em;">050</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>050</mn></math></span></span><script id="MathJax-Element-65" type="math/tex">\$2,050</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;050&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-66-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-820" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-821"><span class="mi" id="MathJax-Span-822" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-823" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-824" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-825" style="font-family: MathJax_Main; padding-left: 0.142em;">050</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>050</mn></math></span></span><script id="MathJax-Element-66" type="math/tex">\$1,050</script></td>
</tr>
<tr>
<td>cluster 1, customer 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;950&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-67-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-826" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-827"><span class="mi" id="MathJax-Span-828" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-829" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-830" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-831" style="font-family: MathJax_Main; padding-left: 0.142em;">950</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>950</mn></math></span></span><script id="MathJax-Element-67" type="math/tex">\$1,950</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;950&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-68-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-832" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-833"><span class="mi" id="MathJax-Span-834" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-835" style="font-family: MathJax_Main;">950</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>950</mn></math></span></span><script id="MathJax-Element-68" type="math/tex">\$950</script></td>
</tr>
<tr>
<td>cluster 1, customer 3</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-69-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-836" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-837"><span class="mi" id="MathJax-Span-838" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-839" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-840" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-841" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-69" type="math/tex">\$2,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-70-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-842" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-843"><span class="mi" id="MathJax-Span-844" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-845" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-846" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-847" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-70" type="math/tex">\$1,000</script></td>
</tr>
<tr>
<td>cluster 1, customer 4</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-71-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-848" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-849"><span class="mi" id="MathJax-Span-850" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-851" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-852" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-853" style="font-family: MathJax_Main; padding-left: 0.142em;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>100</mn></math></span></span><script id="MathJax-Element-71" type="math/tex">\$2,100</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-72-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-854" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-855"><span class="mi" id="MathJax-Span-856" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-857" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-858" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-859" style="font-family: MathJax_Main; padding-left: 0.142em;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>100</mn></math></span></span><script id="MathJax-Element-72" type="math/tex">\$1,100</script></td>
</tr>
<tr>
<td>cluster 1, customer 5</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;900&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-73-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-860" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-861"><span class="mi" id="MathJax-Span-862" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-863" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-864" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-865" style="font-family: MathJax_Main; padding-left: 0.142em;">900</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>900</mn></math></span></span><script id="MathJax-Element-73" type="math/tex">\$1,900</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;900&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-74-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-866" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-867"><span class="mi" id="MathJax-Span-868" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-869" style="font-family: MathJax_Main;">900</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>900</mn></math></span></span><script id="MathJax-Element-74" type="math/tex">\$900</script></td>
</tr>
<tr>
<td>cluster 2, customer 6</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-75-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-870" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-871"><span class="mi" id="MathJax-Span-872" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-873" style="font-family: MathJax_Main;">3</span><span class="mo" id="MathJax-Span-874" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-875" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-75" type="math/tex">\$3,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-76-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-876" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-877"><span class="mi" id="MathJax-Span-878" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-879" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-880" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-881" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-76" type="math/tex">\$2,000</script></td>
</tr>
<tr>
<td>cluster 2, customer 7</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;900&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-77-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-882" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-883"><span class="mi" id="MathJax-Span-884" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-885" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-886" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-887" style="font-family: MathJax_Main; padding-left: 0.142em;">900</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>900</mn></math></span></span><script id="MathJax-Element-77" type="math/tex">\$2,900</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;900&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-78-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-888" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-889"><span class="mi" id="MathJax-Span-890" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-891" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-892" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-893" style="font-family: MathJax_Main; padding-left: 0.142em;">900</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>900</mn></math></span></span><script id="MathJax-Element-78" type="math/tex">\$1,900</script></td>
</tr>
<tr>
<td>cluster 2, customer 8</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;050&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-79-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-894" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-895"><span class="mi" id="MathJax-Span-896" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-897" style="font-family: MathJax_Main;">3</span><span class="mo" id="MathJax-Span-898" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-899" style="font-family: MathJax_Main; padding-left: 0.142em;">050</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3</mn><mo>,</mo><mn>050</mn></math></span></span><script id="MathJax-Element-79" type="math/tex">\$3,050</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;050&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-80-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-900" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-901"><span class="mi" id="MathJax-Span-902" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-903" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-904" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-905" style="font-family: MathJax_Main; padding-left: 0.142em;">050</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>050</mn></math></span></span><script id="MathJax-Element-80" type="math/tex">\$2,050</script></td>
</tr>
<tr>
<td>cluster 2, customer 9</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-81-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-906" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-907"><span class="mi" id="MathJax-Span-908" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-909" style="font-family: MathJax_Main;">3</span><span class="mo" id="MathJax-Span-910" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-911" style="font-family: MathJax_Main; padding-left: 0.142em;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3</mn><mo>,</mo><mn>100</mn></math></span></span><script id="MathJax-Element-81" type="math/tex">\$3,100</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-82-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-912" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-913"><span class="mi" id="MathJax-Span-914" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-915" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-916" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-917" style="font-family: MathJax_Main; padding-left: 0.142em;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>100</mn></math></span></span><script id="MathJax-Element-82" type="math/tex">\$2,100</script></td>
</tr>
<tr>
<td>cluster 2, customer 10</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;950&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-83-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-918" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-919"><span class="mi" id="MathJax-Span-920" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-921" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-922" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-923" style="font-family: MathJax_Main; padding-left: 0.142em;">950</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>950</mn></math></span></span><script id="MathJax-Element-83" type="math/tex">\$2,950</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;950&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-84-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-924" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-925"><span class="mi" id="MathJax-Span-926" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-927" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-928" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-929" style="font-family: MathJax_Main; padding-left: 0.142em;">950</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>950</mn></math></span></span><script id="MathJax-Element-84" type="math/tex">\$1,950</script></td>
</tr>
</tbody>
</table>
<p>The following table shows the cost of offering a product to a customer in a cluster.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1, customer 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;205&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-85-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-930" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-931"><span class="mi" id="MathJax-Span-932" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-933" style="font-family: MathJax_Main;">205</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>205</mn></math></span></span><script id="MathJax-Element-85" type="math/tex">\$205</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;105&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-86-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-934" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-935"><span class="mi" id="MathJax-Span-936" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-937" style="font-family: MathJax_Main;">105</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>105</mn></math></span></span><script id="MathJax-Element-86" type="math/tex">\$105</script></td>
</tr>
<tr>
<td>cluster 1, customer 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;195&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-87-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-938" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-939"><span class="mi" id="MathJax-Span-940" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-941" style="font-family: MathJax_Main;">195</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>195</mn></math></span></span><script id="MathJax-Element-87" type="math/tex">\$195</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;95&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-88-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-942" style="width: 1.878em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.531em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.46em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-943"><span class="mi" id="MathJax-Span-944" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-945" style="font-family: MathJax_Main;">95</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>95</mn></math></span></span><script id="MathJax-Element-88" type="math/tex">\$95</script></td>
</tr>
<tr>
<td>cluster 1, customer 3</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-89-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-946" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-947"><span class="mi" id="MathJax-Span-948" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-949" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-89" type="math/tex">\$200</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-90-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-950" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-951"><span class="mi" id="MathJax-Span-952" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-953" style="font-family: MathJax_Main;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>100</mn></math></span></span><script id="MathJax-Element-90" type="math/tex">\$100</script></td>
</tr>
<tr>
<td>cluster 1, customer 4</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;210&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-91-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-954" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-955"><span class="mi" id="MathJax-Span-956" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-957" style="font-family: MathJax_Main;">210</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>210</mn></math></span></span><script id="MathJax-Element-91" type="math/tex">\$210</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;110&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-92-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-958" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-959"><span class="mi" id="MathJax-Span-960" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-961" style="font-family: MathJax_Main;">110</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>110</mn></math></span></span><script id="MathJax-Element-92" type="math/tex">\$110</script></td>
</tr>
<tr>
<td>cluster 1, customer 5</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;190&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-93-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-962" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-963"><span class="mi" id="MathJax-Span-964" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-965" style="font-family: MathJax_Main;">190</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>190</mn></math></span></span><script id="MathJax-Element-93" type="math/tex">\$190</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;90&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-94-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-966" style="width: 1.878em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.531em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.46em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-967"><span class="mi" id="MathJax-Span-968" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-969" style="font-family: MathJax_Main;">90</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>90</mn></math></span></span><script id="MathJax-Element-94" type="math/tex">\$90</script></td>
</tr>
<tr>
<td>cluster 2, customer 6</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;300&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-95-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-970" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-971"><span class="mi" id="MathJax-Span-972" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-973" style="font-family: MathJax_Main;">300</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>300</mn></math></span></span><script id="MathJax-Element-95" type="math/tex">\$300</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-96-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-974" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-975"><span class="mi" id="MathJax-Span-976" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-977" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-96" type="math/tex">\$200</script></td>
</tr>
<tr>
<td>cluster 2, customer 7</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;290&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-97-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-978" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-979"><span class="mi" id="MathJax-Span-980" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-981" style="font-family: MathJax_Main;">290</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>290</mn></math></span></span><script id="MathJax-Element-97" type="math/tex">\$290</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;190&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-98-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-982" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-983"><span class="mi" id="MathJax-Span-984" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-985" style="font-family: MathJax_Main;">190</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>190</mn></math></span></span><script id="MathJax-Element-98" type="math/tex">\$190</script></td>
</tr>
<tr>
<td>cluster 2, customer 8</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;305&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-99-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-986" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-987"><span class="mi" id="MathJax-Span-988" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-989" style="font-family: MathJax_Main;">305</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>305</mn></math></span></span><script id="MathJax-Element-99" type="math/tex">\$305</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;205&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-100-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-990" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-991"><span class="mi" id="MathJax-Span-992" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-993" style="font-family: MathJax_Main;">205</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>205</mn></math></span></span><script id="MathJax-Element-100" type="math/tex">\$205</script></td>
</tr>
<tr>
<td>cluster 2, customer 9</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;310&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-101-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-994" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-995"><span class="mi" id="MathJax-Span-996" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-997" style="font-family: MathJax_Main;">310</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>310</mn></math></span></span><script id="MathJax-Element-101" type="math/tex">\$310</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;210&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-102-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-998" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-999"><span class="mi" id="MathJax-Span-1000" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1001" style="font-family: MathJax_Main;">210</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>210</mn></math></span></span><script id="MathJax-Element-102" type="math/tex">\$210</script></td>
</tr>
<tr>
<td>cluster 2, customer 10</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;295&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-103-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1002" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1003"><span class="mi" id="MathJax-Span-1004" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1005" style="font-family: MathJax_Main;">295</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>295</mn></math></span></span><script id="MathJax-Element-103" type="math/tex">\$295</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;195&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-104-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1006" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1007"><span class="mi" id="MathJax-Span-1008" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1009" style="font-family: MathJax_Main;">195</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>195</mn></math></span></span><script id="MathJax-Element-104" type="math/tex">\$195</script></td>
</tr>
</tbody>
</table>



<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Python-Implementation">¶</a></h2><p>We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.</p>


In [ ]:

import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

### SETS

products = ['p1', 'p2']
clusters = ['k1', 'k2']




<h3 id="Expected-profit">Expected profit<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Expected-profit">¶</a></h3><p>The following tables shows the expected profit of a customer in each cluster when offered a product.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-105-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1010" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1011"><span class="mi" id="MathJax-Span-1012" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1013" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-1014" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1015" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-105" type="math/tex">\$2,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-106-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1016" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1017"><span class="mi" id="MathJax-Span-1018" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1019" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-1020" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1021" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-106" type="math/tex">\$1,000</script></td>
</tr>
<tr>
<td>cluster 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-107-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1022" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1023"><span class="mi" id="MathJax-Span-1024" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1025" style="font-family: MathJax_Main;">3</span><span class="mo" id="MathJax-Span-1026" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1027" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-107" type="math/tex">\$3,000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-108-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1028" style="width: 3.545em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.85em, 2.573em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1029"><span class="mi" id="MathJax-Span-1030" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1031" style="font-family: MathJax_Main;">2</span><span class="mo" id="MathJax-Span-1032" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1033" style="font-family: MathJax_Main; padding-left: 0.142em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.329em; border-left: 0px solid; width: 0px; height: 1.338em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-108" type="math/tex">\$2,000</script></td>
</tr>
</tbody>
</table>


In [ ]:

### Parameters

# Expected profit
cp, expected_profit = gp.multidict({
    ('k1', 'p1'): 2000,
    ('k1', 'p2'): 1000,
    ('k2', 'p1'): 3000,
    ('k2', 'p2'): 2000
})




<h3 id="Expected-cost">Expected cost<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Expected-cost">¶</a></h3><p>The expected cost of offering a product to a customer in a cluster is shown in the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Product 1</th>
<th>Product 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-109-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1034" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1035"><span class="mi" id="MathJax-Span-1036" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1037" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-109" type="math/tex">\$200</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-110-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1038" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1039"><span class="mi" id="MathJax-Span-1040" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1041" style="font-family: MathJax_Main;">100</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>100</mn></math></span></span><script id="MathJax-Element-110" type="math/tex">\$100</script></td>
</tr>
<tr>
<td>cluster 2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;300&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-111-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1042" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1043"><span class="mi" id="MathJax-Span-1044" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1045" style="font-family: MathJax_Main;">300</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>300</mn></math></span></span><script id="MathJax-Element-111" type="math/tex">\$300</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-112-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1046" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-1047"><span class="mi" id="MathJax-Span-1048" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1049" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-112" type="math/tex">\$200</script></td>
</tr>
</tbody>
</table>


In [ ]:

# Expected cost

cp, expected_cost = gp.multidict({
    ('k1', 'p1'): 200,
    ('k1', 'p2'): 100,
    ('k2', 'p1'): 300,
    ('k2', 'p2'): 200
})




<h3 id="Number-of-customers">Number of customers<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Number-of-customers">¶</a></h3><p>The number of customers in each cluster can be seen in the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Num. Customers</th>
</tr>
</thead>
<tbody>
<tr>
<td>cluster 1</td>
<td>5</td>
</tr>
<tr>
<td>cluster 2</td>
<td>5</td>
</tr>
</tbody>
</table>


In [ ]:

# Num of customers in each cluster

clusters, number_customers = gp.multidict({
    ('k1'): 5,
    ('k2'): 5
})




<h3 id="Minimum-number-of-offers">Minimum number of offers<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Minimum-number-of-offers">¶</a></h3><p>The minimum number of offers of each product is provided in the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Min Offers</th>
</tr>
</thead>
<tbody>
<tr>
<td>product 1</td>
<td>2</td>
</tr>
<tr>
<td>product 2</td>
<td>2</td>
</tr>
</tbody>
</table>


In [ ]:

# Minimum number offers for each product

products, min_offers = gp.multidict({
    ('p1'): 2,
    ('p2'): 2
})




<h3 id="Scalars">Scalars<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Scalars">¶</a></h3><p>The corporate hurdle-rate is twenty percent (<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;R&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;0.20&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-113-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1050" style="width: 4.765em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.932em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.87em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1051"><span class="mi" id="MathJax-Span-1052" style="font-family: MathJax_Math-italic;">R</span><span class="mo" id="MathJax-Span-1053" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-1054" style="font-family: MathJax_Main; padding-left: 0.301em;">0.20</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>R</mi><mo>=</mo><mn>0.20</mn></math></span></span><script id="MathJax-Element-113" type="math/tex">R = 0.20</script>).</p>
<p>The budget available for the marketing campaign is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-114-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1055" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1056"><span class="mi" id="MathJax-Span-1057" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1058" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-114" type="math/tex">\$200</script>.</p>


In [ ]:

# Scalars

R = 0.20

#tight budget
budget = 200 




<h2 id="Tactical-Model-Formulation">Tactical Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Tactical-Model-Formulation">¶</a></h2><h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-115-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1059" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1060"><span class="msubsup" id="MathJax-Span-1061"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1062" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1063"><span class="mrow" id="MathJax-Span-1064"><span class="mi" id="MathJax-Span-1065" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1066" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1067" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1068" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1069" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-115" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-116-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1070" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1071"><span class="mi" id="MathJax-Span-1072" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1073" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1074" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-116" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-117-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1075" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1076"><span class="mi" id="MathJax-Span-1077" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1078" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1079" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-117" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-118-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1080" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.503em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1081"><span class="mi" id="MathJax-Span-1082" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-1083" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1084" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.211em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>z</mi><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-118" type="math/tex">z \geq 0</script>: Increase in budget in order to have a feasible campaign.</p>


In [ ]:

# Declare and initialize model
mt = gp.Model('Tactical')

### Decisions variables

# Allocation of product offers to customers in clusters.

y = mt.addVars(cp, name="allocate")

# Budget correction

z = mt.addVar(name="budget_correction")




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Number of offers</strong>. Maximum number of offers of products for each cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1_2"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-119-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1085" style="width: 100%; display: inline-block; min-width: 14.051em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 14.051em;"><span style="position: absolute; clip: rect(2.443em, 1009.88em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1086"><span class="mtable" id="MathJax-Span-1087" style="min-width: 14.051em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 14.051em;"><span style="display: inline-block; position: absolute; width: 9.884em; height: 0px; clip: rect(-1.545em, 1009.88em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -4.938em;"><span style="position: absolute; clip: rect(2.443em, 1009.88em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.884em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1009.88em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -4.938em;"><span class="mtd" id="MathJax-Span-1091"><span class="mrow" id="MathJax-Span-1092"><span class="munderover" id="MathJax-Span-1093"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1094" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-1095"><span class="mrow" id="MathJax-Span-1096"><span class="mi" id="MathJax-Span-1097" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1098" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1099" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1100" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1101" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1102"><span class="mrow" id="MathJax-Span-1103"><span class="mi" id="MathJax-Span-1104" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1105" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1106" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1107" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1108" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.9em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1109" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-1110"><span class="mrow" id="MathJax-Span-1111"><span class="mi" id="MathJax-Span-1112" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1113" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1114" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1115" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1116" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1117" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1_2"><span class="mrow" id="MathJax-Span-1089"><span class="mtext" id="MathJax-Span-1090" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1_2"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><msub><mi>N</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-119" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-120-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1118" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1119"><span class="msubsup" id="MathJax-Span-1120"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1121" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1122"><span class="mrow" id="MathJax-Span-1123"><span class="mi" id="MathJax-Span-1124" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1125" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1126" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1127" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1128" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-120" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-121-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1129" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1130"><span class="mi" id="MathJax-Span-1131" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1132" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1133" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-121" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-122-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1134" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1135"><span class="mi" id="MathJax-Span-1136" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1137" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1138" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-122" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-123-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1139" style="width: 1.551em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.25em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1140"><span class="msubsup" id="MathJax-Span-1141"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.9em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1142" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-1143"><span class="mrow" id="MathJax-Span-1144"><span class="mi" id="MathJax-Span-1145" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>N</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub></math></span></span><script id="MathJax-Element-123" type="math/tex">N_{k}</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-124-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1146" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1147"><span class="mi" id="MathJax-Span-1148" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1149" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1150" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-124" type="math/tex">k \in K</script>.</p>


In [ ]:

### Constraints

# Constraint on number of offers at each cluster

maxOffers_cons = mt.addConstrs((y.sum(k,'*') <= number_customers[k]  for k in clusters), name='maxOffers')




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Budget</strong>. The marketing campaign budget constraint enforces that the total cost of the campaign should be less than the budget campaign. There is the possibility of increasing the budget to ensure the feasibility of the model, the minimum number of offers for all the product may require this increase in the budget.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2_2"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-125-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1151" style="width: 100%; display: inline-block; min-width: 14.884em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 14.884em;"><span style="position: absolute; clip: rect(2.443em, 1010.78em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1152"><span class="mtable" id="MathJax-Span-1153" style="min-width: 14.884em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 14.884em;"><span style="display: inline-block; position: absolute; width: 10.777em; height: 0px; clip: rect(-1.545em, 1010.78em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -5.354em;"><span style="position: absolute; clip: rect(2.443em, 1010.78em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 10.777em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1010.78em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -5.354em;"><span class="mtd" id="MathJax-Span-1157"><span class="mrow" id="MathJax-Span-1158"><span class="munderover" id="MathJax-Span-1159"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1160" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1161"><span class="mrow" id="MathJax-Span-1162"><span class="mi" id="MathJax-Span-1163" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1164" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1165" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1166" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1167" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-1168"><span class="mrow" id="MathJax-Span-1169"><span class="mi" id="MathJax-Span-1170" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1171" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1172" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1173" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1174" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1175"><span class="mrow" id="MathJax-Span-1176"><span class="mi" id="MathJax-Span-1177" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1178" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1179" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1180" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1181" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1182" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1183"><span class="mrow" id="MathJax-Span-1184"><span class="mi" id="MathJax-Span-1185" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1186" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1187" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1188" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mi" id="MathJax-Span-1189" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">B</span><span class="mo" id="MathJax-Span-1190" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-1191" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-2_2"><span class="mrow" id="MathJax-Span-1155"><span class="mtext" id="MathJax-Span-1156" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2_2"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><mi>B</mi><mo>+</mo><mi>z</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-125" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-126-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1192" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1193"><span class="msubsup" id="MathJax-Span-1194"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1195" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1196"><span class="mrow" id="MathJax-Span-1197"><span class="mi" id="MathJax-Span-1198" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1199" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1200" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1201" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1202" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-126" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-127-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1203" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1204"><span class="mi" id="MathJax-Span-1205" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1206" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1207" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-127" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-128-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1208" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1209"><span class="mi" id="MathJax-Span-1210" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1211" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1212" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-128" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-129-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1213" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.503em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1214"><span class="mi" id="MathJax-Span-1215" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-1216" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1217" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.211em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>z</mi><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-129" type="math/tex">z \geq 0</script>: Increase in budget in order to have a feasible campaign.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-130-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1218" style="width: 1.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.43em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1219"><span class="msubsup" id="MathJax-Span-1220"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1221" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1222"><span class="mrow" id="MathJax-Span-1223"><span class="mi" id="MathJax-Span-1224" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1225" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1226" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-130" type="math/tex">\nu_{k,j}</script>: Average variable cost associated with the offer of product  <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-131-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1227" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1228"><span class="mi" id="MathJax-Span-1229" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1230" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1231" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-131" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-132-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1232" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1233"><span class="mi" id="MathJax-Span-1234" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1235" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1236" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-132" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-133-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1237" style="width: 0.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.78em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1238"><span class="mi" id="MathJax-Span-1239" style="font-family: MathJax_Math-italic;">B</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>B</mi></math></span></span><script id="MathJax-Element-133" type="math/tex">B</script>: Marketing campaign budget.</p>


In [ ]:

# Budget constraint

budget_con = mt.addConstr((y.prod(expected_cost) - z <= budget), name='budget')




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Offers limit</strong>. Minimum number of offers of each product.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3_2"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Q&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-134-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1240" style="width: 100%; display: inline-block; min-width: 13.634em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 13.634em;"><span style="position: absolute; clip: rect(2.443em, 1009.47em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1241"><span class="mtable" id="MathJax-Span-1242" style="min-width: 13.634em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 13.634em;"><span style="display: inline-block; position: absolute; width: 9.467em; height: 0px; clip: rect(-1.545em, 1009.47em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -4.759em;"><span style="position: absolute; clip: rect(2.443em, 1009.47em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1009.47em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -4.759em;"><span class="mtd" id="MathJax-Span-1246"><span class="mrow" id="MathJax-Span-1247"><span class="munderover" id="MathJax-Span-1248"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1249" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1250"><span class="mrow" id="MathJax-Span-1251"><span class="mi" id="MathJax-Span-1252" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1253" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1254" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1255" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1256" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1257"><span class="mrow" id="MathJax-Span-1258"><span class="mi" id="MathJax-Span-1259" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1260" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1261" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1262" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1263" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.72em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1264" style="font-family: MathJax_Math-italic;">Q</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-1265"><span class="mrow" id="MathJax-Span-1266"><span class="mi" id="MathJax-Span-1267" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1268" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1269" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1270" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1271" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1272" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-3_2"><span class="mrow" id="MathJax-Span-1244"><span class="mtext" id="MathJax-Span-1245" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3_2"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>Q</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-134" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-135-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1273" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1274"><span class="msubsup" id="MathJax-Span-1275"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1276" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1277"><span class="mrow" id="MathJax-Span-1278"><span class="mi" id="MathJax-Span-1279" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1280" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1281" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1282" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1283" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-135" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-136-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1284" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1285"><span class="mi" id="MathJax-Span-1286" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1287" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1288" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-136" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-137-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1289" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1290"><span class="mi" id="MathJax-Span-1291" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1292" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1293" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-137" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;Q&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-138-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1294" style="width: 1.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1001.13em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1295"><span class="msubsup" id="MathJax-Span-1296"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.72em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1297" style="font-family: MathJax_Math-italic;">Q</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="texatom" id="MathJax-Span-1298"><span class="mrow" id="MathJax-Span-1299"><span class="mi" id="MathJax-Span-1300" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Q</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-138" type="math/tex">Q_{j}</script>: Minimum number of offers of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-139-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1301" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1302"><span class="mi" id="MathJax-Span-1303" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1304" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1305" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-139" type="math/tex">j \in J</script>.</p>


In [ ]:

# Constraints on min number of offers of each product

minOffers_cons = mt.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>ROI</strong>. The minimum ROI constraint ensures that the ratio of total profits over cost is at least  one plus the corporate hurdle rate.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4_1"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;R&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-140-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1306" style="width: 100%; display: inline-block; min-width: 23.455em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 23.455em;"><span style="position: absolute; clip: rect(2.443em, 1019.29em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1307"><span class="mtable" id="MathJax-Span-1308" style="min-width: 23.455em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 23.455em;"><span style="display: inline-block; position: absolute; width: 19.289em; height: 0px; clip: rect(-1.545em, 1019.29em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -9.64em;"><span style="position: absolute; clip: rect(2.443em, 1019.29em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 19.289em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1019.29em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -9.64em;"><span class="mtd" id="MathJax-Span-1312"><span class="mrow" id="MathJax-Span-1313"><span class="munderover" id="MathJax-Span-1314"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1315" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1316"><span class="mrow" id="MathJax-Span-1317"><span class="mi" id="MathJax-Span-1318" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1319" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1320" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1321" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1322" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-1323"><span class="mrow" id="MathJax-Span-1324"><span class="mi" id="MathJax-Span-1325" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1326" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1327" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1328" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1329" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1330"><span class="mrow" id="MathJax-Span-1331"><span class="mi" id="MathJax-Span-1332" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1333" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1334" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1335" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1336" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1337" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1338"><span class="mrow" id="MathJax-Span-1339"><span class="mi" id="MathJax-Span-1340" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1341" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1342" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1343" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mo" id="MathJax-Span-1344" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="mn" id="MathJax-Span-1345" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-1346" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-1347" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">R</span><span class="mo" id="MathJax-Span-1348" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-1349" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1350" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1351" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1352"><span class="mrow" id="MathJax-Span-1353"><span class="mi" id="MathJax-Span-1354" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1355" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1356" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1357" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1358" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-1359"><span class="mrow" id="MathJax-Span-1360"><span class="mi" id="MathJax-Span-1361" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1362" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1363" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1364" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1365" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1366"><span class="mrow" id="MathJax-Span-1367"><span class="mi" id="MathJax-Span-1368" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1369" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1370" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1371" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1372" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1373" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1374"><span class="mrow" id="MathJax-Span-1375"><span class="mi" id="MathJax-Span-1376" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1377" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1378" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-4_1"><span class="mrow" id="MathJax-Span-1310"><span class="mtext" id="MathJax-Span-1311" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4_1"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mo stretchy="false">(</mo><mn>1</mn><mo>+</mo><mi>R</mi><mo stretchy="false">)</mo><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-140" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-141-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1379" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1380"><span class="msubsup" id="MathJax-Span-1381"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1382" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1383"><span class="mrow" id="MathJax-Span-1384"><span class="mi" id="MathJax-Span-1385" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1386" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1387" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1388" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1389" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-141" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-142-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1390" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1391"><span class="mi" id="MathJax-Span-1392" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1393" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1394" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-142" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-143-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1395" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1396"><span class="mi" id="MathJax-Span-1397" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1398" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1399" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-143" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-144-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1400" style="width: 1.789em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1401"><span class="msubsup" id="MathJax-Span-1402"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1403" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1404"><span class="mrow" id="MathJax-Span-1405"><span class="mi" id="MathJax-Span-1406" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1407" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1408" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-144" type="math/tex">\pi_{k,j}</script>: Expected profit to the bank from the offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-145-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1409" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1410"><span class="mi" id="MathJax-Span-1411" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1412" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1413" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-145" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-146-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1414" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1415"><span class="mi" id="MathJax-Span-1416" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1417" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1418" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-146" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03BD;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-147-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1419" style="width: 1.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.43em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1420"><span class="msubsup" id="MathJax-Span-1421"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1422" style="font-family: MathJax_Math-italic;">ν<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1423"><span class="mrow" id="MathJax-Span-1424"><span class="mi" id="MathJax-Span-1425" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1426" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1427" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>ν</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-147" type="math/tex">\nu_{k,j}</script>: Average variable cost associated with the offer of product  <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-148-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1428" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1429"><span class="mi" id="MathJax-Span-1430" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1431" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1432" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-148" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-149-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1433" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1434"><span class="mi" id="MathJax-Span-1435" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1436" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1437" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-149" type="math/tex">k \in K</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;R&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-150-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1438" style="width: 0.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.78em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1439"><span class="mi" id="MathJax-Span-1440" style="font-family: MathJax_Math-italic;">R</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>R</mi></math></span></span><script id="MathJax-Element-150" type="math/tex">R</script>: Corporate hurdle rate.</p>


In [ ]:

# Constraint to ensure minimum ROI

ROI_con = mt.addConstr((y.prod(expected_profit) - (1 + R)*y.prod(expected_cost) >= 0), name='ROI')




<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-Function">¶</a></h3><ul>
<li><strong>Total profit</strong>. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0_2"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-151-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1441" style="width: 100%; display: inline-block; min-width: 19.705em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 19.705em;"><span style="position: absolute; clip: rect(2.443em, 1015.54em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1442"><span class="mtable" id="MathJax-Span-1443" style="min-width: 19.705em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 19.705em;"><span style="display: inline-block; position: absolute; width: 15.539em; height: 0px; clip: rect(-1.545em, 1015.54em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.795em;"><span style="position: absolute; clip: rect(2.443em, 1015.54em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 15.539em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1015.54em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.795em;"><span class="mtd" id="MathJax-Span-1447"><span class="mrow" id="MathJax-Span-1448"><span class="mtext" id="MathJax-Span-1449" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-1450" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1451" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1452" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-1453" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1454" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1455"><span class="mrow" id="MathJax-Span-1456"><span class="mi" id="MathJax-Span-1457" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1458" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1459" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1460" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1461" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-1462"><span class="mrow" id="MathJax-Span-1463"><span class="mi" id="MathJax-Span-1464" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1465" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1466" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1467" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1468" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1469"><span class="mrow" id="MathJax-Span-1470"><span class="mi" id="MathJax-Span-1471" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1472" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1473" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1474" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1475" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1476" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1477"><span class="mrow" id="MathJax-Span-1478"><span class="mi" id="MathJax-Span-1479" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1480" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1481" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1482" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-1483" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1484" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-1485" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-0_2"><span class="mrow" id="MathJax-Span-1445"><span class="mtext" id="MathJax-Span-1446" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0_2"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>−</mo><mi>M</mi><mo>⋅</mo><mi>z</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-151" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-152-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1486" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1487"><span class="msubsup" id="MathJax-Span-1488"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1489" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1490"><span class="mrow" id="MathJax-Span-1491"><span class="mi" id="MathJax-Span-1492" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1493" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1494" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1495" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1496" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-152" type="math/tex">y_{k,j} \geq 0</script>: Number of customers in cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-153-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1497" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1498"><span class="mi" id="MathJax-Span-1499" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1500" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1501" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-153" type="math/tex">k \in K</script> that are offered product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-154-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1502" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1503"><span class="mi" id="MathJax-Span-1504" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1505" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1506" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-154" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-155-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1507" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.503em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1508"><span class="mi" id="MathJax-Span-1509" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-1510" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1511" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.211em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>z</mi><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-155" type="math/tex">z \geq 0</script>: Increase in budget in order to have a feasible campaign.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-156-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1512" style="width: 1.789em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1513"><span class="msubsup" id="MathJax-Span-1514"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1515" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1516"><span class="mrow" id="MathJax-Span-1517"><span class="mi" id="MathJax-Span-1518" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1519" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1520" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-156" type="math/tex">\pi_{k,j}</script>: Expected profit to the bank from the offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-157-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1521" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1522"><span class="mi" id="MathJax-Span-1523" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1524" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1525" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-157" type="math/tex">j \in J</script> to an average customer of cluster <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-158-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1526" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1527"><span class="mi" id="MathJax-Span-1528" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1529" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1530" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-158" type="math/tex">k \in K</script>.</p>
<p><strong>Note:</strong> The value of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-159-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1531" style="width: 1.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.01em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1532"><span class="mi" id="MathJax-Span-1533" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi></math></span></span><script id="MathJax-Element-159" type="math/tex">M</script> should be higher than any of the expected profits to ensure that the budget is increased only when the model is infeasible if this parameter is not increased.</p>


In [ ]:

### Objective function

# Maximize total expected profit

M = 10000

mt.setObjective(y.prod(expected_profit) -M*z, GRB.MAXIMIZE)



In [ ]:

# Verify model formulation

mt.write('tactical.lp')



In [ ]:

# Run optimization engine

mt.optimize()



In [ ]:

### Output Reports

# Optimal allocation of product offers to clusters

total_expected_profit = 0
total_expected_cost = 0

print("\nOptimal allocation of product offers to clusters.")
print("___________________________________________________")
for k,p in cp:
    if y[k,p].x > 1e-6:
        #print(y[k,p].varName, y[k,p].x)
        print(f"The number of customers in cluster {k} that gets an offer of product {p} is: {y[k,p].x}")
        total_expected_profit += expected_profit[k,p]*y[k,p].x
        total_expected_cost += expected_cost[k,p]*y[k,p].x

increased_budget = '${:,.2f}'.format(z.x)
print(f"\nThe increase correction in the campaign budget is {increased_budget}.")

# Financial reports

optimal_ROI = round(100*total_expected_profit/total_expected_cost,2)
min_ROI = round(100*(1+R),2)

money_expected_profit = '${:,.2f}'.format(total_expected_profit)
money_expected_cost = '${:,.2f}'.format(total_expected_cost)
money_budget = '${:,.2f}'.format(budget)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total expected profit is {money_expected_profit}.")
print(f"Optimal total expected cost is {money_expected_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {optimal_ROI}% with a minimum ROI of  {min_ROI}%.")




<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Analysis">¶</a></h2><p>The cost of allocating products to clusters required an increase in the budget of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;400&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-160-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1534" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1535"><span class="mi" id="MathJax-Span-1536" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1537" style="font-family: MathJax_Main;">400</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>400</mn></math></span></span><script id="MathJax-Element-160" type="math/tex">\$400</script>. The total expected profit is 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;6&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-161-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1538" style="width: 3.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1002.92em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1539"><span class="mi" id="MathJax-Span-1540" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1541" style="font-family: MathJax_Main;">6</span><span class="mo" id="MathJax-Span-1542" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1543" style="font-family: MathJax_Main; padding-left: 0.182em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>6</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-161" type="math/tex">\$6,000</script>. The total expected cost is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;600&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-162-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1544" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1545"><span class="mi" id="MathJax-Span-1546" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1547" style="font-family: MathJax_Main;">600</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>600</mn></math></span></span><script id="MathJax-Element-162" type="math/tex">\$600</script> which is equal to the original budget of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-163-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1548" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1549"><span class="mi" id="MathJax-Span-1550" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1551" style="font-family: MathJax_Main;">200</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>200</mn></math></span></span><script id="MathJax-Element-163" type="math/tex">\$200</script> plus the increase of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;400&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-164-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1552" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1553"><span class="mi" id="MathJax-Span-1554" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1555" style="font-family: MathJax_Main;">400</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>400</mn></math></span></span><script id="MathJax-Element-164" type="math/tex">\$400</script>. The expected ROI is 1,000 % which is much higher than the minimum ROI required.</p>



<h2 id="Operational-Model-Formulation">Operational Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Operational-Model-Formulation">¶</a></h2><h3 id="Customer-expected-profit">Customer expected profit<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Customer-expected-profit">¶</a></h3>


In [ ]:

### Sets

customers = ['c1', 'c2','c3','c4','c5','c6','c7','c8','c9','c10']

### Parameters

# Expected profit from a product offering for each customer in each cluster
ccp, customer_profit = gp.multidict({
    ('k1', 'c1', 'p1'): 2050,
    ('k1', 'c1', 'p2'): 1050,
    ('k1', 'c2', 'p1'): 1950,
    ('k1', 'c2', 'p2'): 950,
    ('k1', 'c3', 'p1'): 2000,
    ('k1', 'c3', 'p2'): 1000,
    ('k1', 'c4', 'p1'): 2100,
    ('k1', 'c4', 'p2'): 1100,
    ('k1', 'c5', 'p1'): 1900,
    ('k1', 'c5', 'p2'): 900,
    ('k2', 'c6', 'p1'): 3000,
    ('k2', 'c6', 'p2'): 2000,
    ('k2', 'c7', 'p1'): 2900,
    ('k2', 'c7', 'p2'): 1900,
    ('k2', 'c8', 'p1'): 3050,
    ('k2', 'c8','p2'): 2050,
    ('k2', 'c9', 'p1'): 3100,
    ('k2', 'c9', 'p2'): 3100,
    ('k2', 'c10', 'p1'): 2950,
    ('k2', 'c10', 'p2'): 2950   
})




<h3 id="Customer-offering-cost">Customer offering cost<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Customer-offering-cost">¶</a></h3>


In [ ]:

# Customer cost of offering a product at a cluster

ccp, customer_cost = gp.multidict({
    ('k1', 'c1', 'p1'): 205,
    ('k1', 'c1', 'p2'): 105,
    ('k1', 'c2', 'p1'): 195,
    ('k1', 'c2', 'p2'): 95,
    ('k1', 'c3', 'p1'): 200,
    ('k1', 'c3', 'p2'): 100,
    ('k1', 'c4', 'p1'): 210,
    ('k1', 'c4', 'p2'): 110,
    ('k1', 'c5', 'p1'): 190,
    ('k1', 'c5', 'p2'): 90,
    ('k2', 'c6', 'p1'): 300,
    ('k2', 'c6', 'p2'): 200,
    ('k2', 'c7', 'p1'): 290,
    ('k2', 'c7', 'p2'): 190,
    ('k2', 'c8', 'p1'): 305,
    ('k2', 'c8','p2'): 205,
    ('k2', 'c9', 'p1'): 310,
    ('k2', 'c9', 'p2'): 310,
    ('k2', 'c10', 'p1'): 295,
    ('k2', 'c10', 'p2'): 295   
})




<h2 id="Operational-Model-Formulation">Operational Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Operational-Model-Formulation">¶</a></h2>



<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-165-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1556" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1557"><span class="msubsup" id="MathJax-Span-1558"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1559" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1560"><span class="mrow" id="MathJax-Span-1561"><span class="mi" id="MathJax-Span-1562" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1563" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1564" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1565" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1566" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1567" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-1568" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-1569" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-1570" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1571" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-1572" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-165" type="math/tex">x_{k,i,j} \in \{0,1 \}</script>: This variable is equal to 1, if product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-166-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1573" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1574"><span class="mi" id="MathJax-Span-1575" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1576" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1577" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1578" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1579"><span class="mrow" id="MathJax-Span-1580"><span class="mi" id="MathJax-Span-1581" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-166" type="math/tex">j \in J^{k}</script>  is offered to customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-167-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1582" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1583"><span class="mi" id="MathJax-Span-1584" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1585" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1586" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1587" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1588"><span class="mrow" id="MathJax-Span-1589"><span class="mi" id="MathJax-Span-1590" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-167" type="math/tex">i \in I^{k}</script>, and 0 otherwise.</p>


In [ ]:

# Declare and initialize model
mo = gp.Model('Operational')

### Decision variables

x = mo.addVars(ccp, vtype=GRB.BINARY, name="assign")




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Product offers</strong>. Allocate offers of a product to customers of each cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1_3"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-168-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1591" style="width: 100%; display: inline-block; min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.158em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1592"><span class="mtable" id="MathJax-Span-1593" style="min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.158em;"><span style="display: inline-block; position: absolute; width: 13.991em; height: 0px; clip: rect(-1.604em, 1013.99em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.021em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.991em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.99em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.021em;"><span class="mtd" id="MathJax-Span-1597"><span class="mrow" id="MathJax-Span-1598"><span class="munderover" id="MathJax-Span-1599"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1600" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1601"><span class="mrow" id="MathJax-Span-1602"><span class="mi" id="MathJax-Span-1603" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1604" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1605"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1606" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1607"><span class="mrow" id="MathJax-Span-1608"><span class="mi" id="MathJax-Span-1609" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1610" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1611" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1612"><span class="mrow" id="MathJax-Span-1613"><span class="mi" id="MathJax-Span-1614" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1615" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1616" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1617" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1618" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1619" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-1620" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1621" style="font-family: MathJax_Math-italic;">Y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1622"><span class="mrow" id="MathJax-Span-1623"><span class="mi" id="MathJax-Span-1624" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1625" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1626" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1627" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1628" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1629" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1630" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1631" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1632" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1633"><span class="mrow" id="MathJax-Span-1634"><span class="mi" id="MathJax-Span-1635" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1636" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1637" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-1638" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1639" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, -0.057em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1_3"><span class="mrow" id="MathJax-Span-1595"><span class="mtext" id="MathJax-Span-1596" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1_3"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>=</mo><msub><mi>Y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-168" type="math/tex; mode=display">\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-169-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1640" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1641"><span class="msubsup" id="MathJax-Span-1642"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1643" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1644"><span class="mrow" id="MathJax-Span-1645"><span class="mi" id="MathJax-Span-1646" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1647" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1648" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1649" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1650" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1651" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-1652" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-1653" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-1654" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1655" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-1656" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-169" type="math/tex">x_{k,i,j} \in \{0,1 \}</script>: This variable is equal to 1, if product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-170-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1657" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1658"><span class="mi" id="MathJax-Span-1659" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1660" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1661" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1662" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1663"><span class="mrow" id="MathJax-Span-1664"><span class="mi" id="MathJax-Span-1665" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-170" type="math/tex">j \in J^{k}</script>  is offered to customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-171-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1666" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1667"><span class="mi" id="MathJax-Span-1668" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1669" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1670" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1671" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1672"><span class="mrow" id="MathJax-Span-1673"><span class="mi" id="MathJax-Span-1674" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-171" type="math/tex">i \in I^{k}</script>, and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;Y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;&amp;#x230A;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo fence="false" stretchy="false"&gt;&amp;#x230B;&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-172-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1675" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1676"><span class="msubsup" id="MathJax-Span-1677"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1678" style="font-family: MathJax_Math-italic;">Y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1679"><span class="mrow" id="MathJax-Span-1680"><span class="mi" id="MathJax-Span-1681" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1682" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1683" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1684" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-1685" style="font-family: MathJax_Main; padding-left: 0.301em;">⌊</span><span class="msubsup" id="MathJax-Span-1686"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1687" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1688"><span class="mrow" id="MathJax-Span-1689"><span class="mi" id="MathJax-Span-1690" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1691" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1692" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1693" style="font-family: MathJax_Main;">⌋</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>=</mo><mo fence="false" stretchy="false">⌊</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mo fence="false" stretchy="false">⌋</mo></math></span></span><script id="MathJax-Element-172" type="math/tex">Y_{k,j} = \lfloor y_{k,j} \rfloor </script>: Number of customers in cluster k that will get an offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-173-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1694" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1695"><span class="mi" id="MathJax-Span-1696" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1697" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1698" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1699" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1700"><span class="mrow" id="MathJax-Span-1701"><span class="mi" id="MathJax-Span-1702" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-173" type="math/tex">j \in J^{k}</script>.</p>


In [ ]:

# Product offers constraint

productOffers = {}

for k in clusters:
    for j in products:
            productOffers[k,j] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffers_' + str(k) + ',' + str(j) )




<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Offers limit</strong>. At most one product may be offered to a customer of a cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2_3"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-174-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1703" style="width: 100%; display: inline-block; min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.086em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1704"><span class="mtable" id="MathJax-Span-1705" style="min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.086em;"><span style="display: inline-block; position: absolute; width: 12.92em; height: 0px; clip: rect(-1.664em, 1012.92em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -6.485em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 12.92em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1012.92em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.485em;"><span class="mtd" id="MathJax-Span-1709"><span class="mrow" id="MathJax-Span-1710"><span class="munderover" id="MathJax-Span-1711"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-1712" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1713"><span class="mrow" id="MathJax-Span-1714"><span class="mi" id="MathJax-Span-1715" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1716" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1717"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1718" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1719"><span class="mrow" id="MathJax-Span-1720"><span class="mi" id="MathJax-Span-1721" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1722" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1723" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1724"><span class="mrow" id="MathJax-Span-1725"><span class="mi" id="MathJax-Span-1726" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1727" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1728" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1729" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1730" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1731" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-1732" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-1733" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1734" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1735" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1736" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1737" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1738" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1739"><span class="mrow" id="MathJax-Span-1740"><span class="mi" id="MathJax-Span-1741" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1742" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1743" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-1744" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1745" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-2_3"><span class="mrow" id="MathJax-Span-1707"><span class="mtext" id="MathJax-Span-1708" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2_3"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-174" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-175-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1746" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1747"><span class="msubsup" id="MathJax-Span-1748"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1749" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1750"><span class="mrow" id="MathJax-Span-1751"><span class="mi" id="MathJax-Span-1752" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1753" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1754" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1755" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1756" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1757" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-1758" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-1759" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-1760" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1761" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-1762" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-175" type="math/tex">x_{k,i,j} \in \{0,1 \}</script>: This variable is equal to 1, if product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-176-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1763" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1764"><span class="mi" id="MathJax-Span-1765" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1766" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1767" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1768" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1769"><span class="mrow" id="MathJax-Span-1770"><span class="mi" id="MathJax-Span-1771" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-176" type="math/tex">j \in J^{k}</script>  is offered to customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-177-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1772" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1773"><span class="mi" id="MathJax-Span-1774" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1775" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1776" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1777" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1778"><span class="mrow" id="MathJax-Span-1779"><span class="mi" id="MathJax-Span-1780" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-177" type="math/tex">i \in I^{k}</script>, and 0 otherwise.</p>


In [ ]:

# limit on the number of offers to each customer in a cluster.

ki = [('k1', 'c1'), 
      ('k1', 'c2'), 
      ('k1', 'c3'),
      ('k1', 'c4'), 
      ('k1', 'c5'), 
      ('k2', 'c6'), 
      ('k2', 'c7'), 
      ('k2', 'c8'), 
      ('k2', 'c9'), 
      ('k2', 'c10')]

customerOffers = {}

for k,i in ki:
    customerOffers[k,i] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffers_' + str(k) + ',' + str(i) )




<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-Function">¶</a></h3><ul>
<li><strong>Total profit</strong>. Maximize total individual expected profit.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0_3"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-178-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1781" style="width: 100%; display: inline-block; min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.813em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1782"><span class="mtable" id="MathJax-Span-1783" style="min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.813em;"><span style="display: inline-block; position: absolute; width: 14.646em; height: 0px; clip: rect(-1.664em, 1014.65em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -7.318em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 14.646em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1014.65em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -7.318em;"><span class="mtd" id="MathJax-Span-1787"><span class="mrow" id="MathJax-Span-1788"><span class="mtext" id="MathJax-Span-1789" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-1790" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1791" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1792" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-1793" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1794" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1795"><span class="mrow" id="MathJax-Span-1796"><span class="mi" id="MathJax-Span-1797" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1798" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1799" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1800" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1801" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1802"><span class="mrow" id="MathJax-Span-1803"><span class="mi" id="MathJax-Span-1804" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1805" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1806"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1807" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1808"><span class="mrow" id="MathJax-Span-1809"><span class="mi" id="MathJax-Span-1810" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1811" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-1812" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1813"><span class="mrow" id="MathJax-Span-1814"><span class="mi" id="MathJax-Span-1815" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1816" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1817"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1818" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1819"><span class="mrow" id="MathJax-Span-1820"><span class="mi" id="MathJax-Span-1821" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1822" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1823" style="font-family: MathJax_Math-italic;">r</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1824"><span class="mrow" id="MathJax-Span-1825"><span class="mi" id="MathJax-Span-1826" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1827" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1828" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1829" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1830" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1831" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1832" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1833" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1834"><span class="mrow" id="MathJax-Span-1835"><span class="mi" id="MathJax-Span-1836" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1837" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1838" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1839" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1840" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-0_3"><span class="mrow" id="MathJax-Span-1785"><span class="mtext" id="MathJax-Span-1786" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0_3"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>r</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-178" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}</script><p>Where</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-179-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1841" style="width: 6.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1842"><span class="msubsup" id="MathJax-Span-1843"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1844" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1845"><span class="mrow" id="MathJax-Span-1846"><span class="mi" id="MathJax-Span-1847" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1848" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1849" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1850" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1851" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1852" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-1853" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-1854" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-1855" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1856" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-1857" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-179" type="math/tex">x_{k,i,j} \in \{0,1 \}</script>: This variable is equal to 1, if product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-180-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1858" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1859"><span class="mi" id="MathJax-Span-1860" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1861" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1862" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1863" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1864"><span class="mrow" id="MathJax-Span-1865"><span class="mi" id="MathJax-Span-1866" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-180" type="math/tex">j \in J^{k}</script>  is offered to customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-181-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1867" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1868"><span class="mi" id="MathJax-Span-1869" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1870" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1871" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1872" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1873"><span class="mrow" id="MathJax-Span-1874"><span class="mi" id="MathJax-Span-1875" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-181" type="math/tex">i \in I^{k}</script>, and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-182-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1876" style="width: 2.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1001.85em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1877"><span class="msubsup" id="MathJax-Span-1878"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1879" style="font-family: MathJax_Math-italic;">r</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1880"><span class="mrow" id="MathJax-Span-1881"><span class="mi" id="MathJax-Span-1882" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1883" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1884" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1885" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1886" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>r</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-182" type="math/tex">r_{k,i,j}</script>: Expected individual profit of customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-183-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1887" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1888"><span class="mi" id="MathJax-Span-1889" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1890" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1891" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1892" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1893"><span class="mrow" id="MathJax-Span-1894"><span class="mi" id="MathJax-Span-1895" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-183" type="math/tex">i \in I^{k}</script> from  offer of product <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-184-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1896" style="width: 3.396em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1002.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1897"><span class="mi" id="MathJax-Span-1898" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1899" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-1900" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1901" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-1902"><span class="mrow" id="MathJax-Span-1903"><span class="mi" id="MathJax-Span-1904" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></math></span></span><script id="MathJax-Element-184" type="math/tex">j \in J^{k}</script>.</p>


In [ ]:

### Objective function

# Maximoze total profit

mo.setObjective(x.prod(customer_profit), GRB.MAXIMIZE)



In [ ]:

# Verify model formulation

mo.write('operational.lp')

# Run optimization engine

mo.optimize()



In [ ]:

### Output Reports

# Optimal assignment of product offers to customers

total_customer_profit = 0
total_customer_cost = 0

kvalue = None
first = True
num_assignments = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalue:
        prevk = kvalue
        kvalue = k
        if not first:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevk} is {num_assignments}")
            print("___________________________________________________")
            num_assignments = 0
        if first:
            first = False
    if x[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profit = '${:,.2f}'.format(customer_profit[k,i,j])
        cost = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profit} at a cost of {cost}")
        total_customer_profit += customer_profit[k,i,j]*x[k,i,j].x
        total_customer_cost += customer_cost[k,i,j]*x[k,i,j].x
        num_assignments += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalue} is {num_assignments}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROI = round(100*total_customer_profit/total_customer_cost,2)

money_customers_profit = '${:,.2f}'.format(total_customer_profit)
money_customers_cost = '${:,.2f}'.format(total_customer_cost)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profit}.")
print(f"Optimal total customers cost is {money_customers_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROI}% with a minimum ROI of  {min_ROI}%.")
        




<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Analysis">¶</a></h2><p>Each customer got, at most, one product offer.  Product p2 is offered to customers c1 and c2, and product p1 is offered to customers c3 and c4. Products p1 and p2 are offerred to at least two customers -this is a constraint from the tactical model. Observe that to ensure these hard business constraints, the budget needs to be increased by <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;400&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-185-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1905" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1906"><span class="mi" id="MathJax-Span-1907" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1908" style="font-family: MathJax_Main;">400</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>400</mn></math></span></span><script id="MathJax-Element-185" type="math/tex">\$400</script></p>
<p>The cost of assigning  products to customers is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;610&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-186-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1909" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1910"><span class="mi" id="MathJax-Span-1911" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1912" style="font-family: MathJax_Main;">610</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>610</mn></math></span></span><script id="MathJax-Element-186" type="math/tex">\$610</script>, which slightly violates the total budget available of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;600&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-187-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1913" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1914"><span class="mi" id="MathJax-Span-1915" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1916" style="font-family: MathJax_Main;">600</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>600</mn></math></span></span><script id="MathJax-Element-187" type="math/tex">\$600</script>. The total customers profit is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;6&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;100&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-188-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1917" style="width: 3.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1002.92em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1918"><span class="mi" id="MathJax-Span-1919" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1920" style="font-family: MathJax_Main;">6</span><span class="mo" id="MathJax-Span-1921" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1922" style="font-family: MathJax_Main; padding-left: 0.182em;">100</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>6</mn><mo>,</mo><mn>100</mn></math></span></span><script id="MathJax-Element-188" type="math/tex">\$6,100</script>. The  ROI is 1,000 %, which is much higher than the minimum ROI required.</p>
<p>If the total available budget needs to be enforced, the following constraint can be added to the operational model:</p>
<ul>
<li><strong>Budget</strong>. Enforce budget constraint.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4_2"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;&amp;#x2032;&lt;/mo&gt;&lt;/msup&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-189-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1923" style="width: 100%; display: inline-block; min-width: 16.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 16.134em;"><span style="position: absolute; clip: rect(2.324em, 1011.97em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1924"><span class="mtable" id="MathJax-Span-1925" style="min-width: 16.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 16.134em;"><span style="display: inline-block; position: absolute; width: 11.967em; height: 0px; clip: rect(-1.664em, 1011.97em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -6.009em;"><span style="position: absolute; clip: rect(2.324em, 1011.97em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 11.967em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1011.97em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.009em;"><span class="mtd" id="MathJax-Span-1929"><span class="mrow" id="MathJax-Span-1930"><span class="munderover" id="MathJax-Span-1931"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1932" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1933"><span class="mrow" id="MathJax-Span-1934"><span class="mi" id="MathJax-Span-1935" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1936" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1937" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1938" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-1939" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1940"><span class="mrow" id="MathJax-Span-1941"><span class="mi" id="MathJax-Span-1942" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1943" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1944"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1945" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1946"><span class="mrow" id="MathJax-Span-1947"><span class="mi" id="MathJax-Span-1948" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-1949" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-1950" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1951"><span class="mrow" id="MathJax-Span-1952"><span class="mi" id="MathJax-Span-1953" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1954" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-1955"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1956" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1957"><span class="mrow" id="MathJax-Span-1958"><span class="mi" id="MathJax-Span-1959" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1960" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.789em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1961" style="font-family: MathJax_Math-italic;">c</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1962"><span class="mrow" id="MathJax-Span-1963"><span class="mi" id="MathJax-Span-1964" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1965" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1966" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1967" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1968" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1969" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1970" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1971" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1972"><span class="mrow" id="MathJax-Span-1973"><span class="mi" id="MathJax-Span-1974" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1975" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1976" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1977" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1978" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1979" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msup" id="MathJax-Span-1980" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1981" style="font-family: MathJax_Math-italic;">B</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.777em;"><span class="mo" id="MathJax-Span-1982" style="font-size: 70.7%; font-family: MathJax_Main;">′</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-4_2"><span class="mrow" id="MathJax-Span-1927"><span class="mtext" id="MathJax-Span-1928" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4_2"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>c</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><msup><mi>B</mi><mo>′</mo></msup></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-189" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{4}
\end{equation}</script><p>The new budget is the original budget plus the correction, that is  <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msup&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;&amp;#x2032;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-190-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1983" style="width: 5.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.884em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.88em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1984"><span class="msup" id="MathJax-Span-1985"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1986" style="font-family: MathJax_Math-italic;">B</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.777em;"><span class="mo" id="MathJax-Span-1987" style="font-size: 70.7%; font-family: MathJax_Main;">′</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1988" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-1989" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">B</span><span class="mo" id="MathJax-Span-1990" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-1991" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msup><mi>B</mi><mo>′</mo></msup><mo>=</mo><mi>B</mi><mo>+</mo><mi>z</mi></math></span></span><script id="MathJax-Element-190" type="math/tex">B' = B + z</script></p>



<h2 id="Scenario-1">Scenario 1<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Scenario-1">¶</a></h2><p>Enforce total budget available constraint. In this case, the operational model is:</p>
<h3 id="Objective-function">Objective function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Objective-function">¶</a></h3><ul>
<li><strong>Total profit</strong>. Maximize total individual expected profit.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0_4"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-191-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1992" style="width: 100%; display: inline-block; min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.813em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1993"><span class="mtable" id="MathJax-Span-1994" style="min-width: 18.813em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.813em;"><span style="display: inline-block; position: absolute; width: 14.646em; height: 0px; clip: rect(-1.664em, 1014.65em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -7.318em;"><span style="position: absolute; clip: rect(2.324em, 1014.65em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 14.646em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1014.65em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -7.318em;"><span class="mtd" id="MathJax-Span-1998"><span class="mrow" id="MathJax-Span-1999"><span class="mtext" id="MathJax-Span-2000" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-2001" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-2002" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-2003" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-2004" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-2005" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-2006"><span class="mrow" id="MathJax-Span-2007"><span class="mi" id="MathJax-Span-2008" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2009" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-2010" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-2011" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-2012" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2013"><span class="mrow" id="MathJax-Span-2014"><span class="mi" id="MathJax-Span-2015" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2016" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2017"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2018" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-2019"><span class="mrow" id="MathJax-Span-2020"><span class="mi" id="MathJax-Span-2021" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-2022" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-2023" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2024"><span class="mrow" id="MathJax-Span-2025"><span class="mi" id="MathJax-Span-2026" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-2027" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2028"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2029" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-2030"><span class="mrow" id="MathJax-Span-2031"><span class="mi" id="MathJax-Span-2032" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-2033" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2034" style="font-family: MathJax_Math-italic;">r</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-2035"><span class="mrow" id="MathJax-Span-2036"><span class="mi" id="MathJax-Span-2037" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2038" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2039" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2040" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2041" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2042" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-2043" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2044" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2045"><span class="mrow" id="MathJax-Span-2046"><span class="mi" id="MathJax-Span-2047" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2048" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2049" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2050" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2051" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-0_4"><span class="mrow" id="MathJax-Span-1996"><span class="mtext" id="MathJax-Span-1997" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0_4"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>r</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-191" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Constraints">¶</a></h3><ul>
<li><strong>Product offers</strong>. Allocate offers of a product to customers of each cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1_4"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-192-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-2052" style="width: 100%; display: inline-block; min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.158em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-2053"><span class="mtable" id="MathJax-Span-2054" style="min-width: 18.158em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.158em;"><span style="display: inline-block; position: absolute; width: 13.991em; height: 0px; clip: rect(-1.604em, 1013.99em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.021em;"><span style="position: absolute; clip: rect(2.384em, 1013.99em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.991em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.99em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.021em;"><span class="mtd" id="MathJax-Span-2058"><span class="mrow" id="MathJax-Span-2059"><span class="munderover" id="MathJax-Span-2060"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-2061" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2062"><span class="mrow" id="MathJax-Span-2063"><span class="mi" id="MathJax-Span-2064" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2065" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2066"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2067" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-2068"><span class="mrow" id="MathJax-Span-2069"><span class="mi" id="MathJax-Span-2070" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-2071" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2072" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2073"><span class="mrow" id="MathJax-Span-2074"><span class="mi" id="MathJax-Span-2075" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2076" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2077" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2078" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2079" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2080" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-2081" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2082" style="font-family: MathJax_Math-italic;">Y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2083"><span class="mrow" id="MathJax-Span-2084"><span class="mi" id="MathJax-Span-2085" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2086" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2087" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-2088" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-2089" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-2090" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-2091" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-2092" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2093" style="font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="texatom" id="MathJax-Span-2094"><span class="mrow" id="MathJax-Span-2095"><span class="mi" id="MathJax-Span-2096" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2097" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2098" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-2099" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-2100" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, -0.057em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1_4"><span class="mrow" id="MathJax-Span-2056"><span class="mtext" id="MathJax-Span-2057" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1_4"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>=</mo><msub><mi>Y</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-192" type="math/tex; mode=display">\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}</script><ul>
<li><strong>Offers limit</strong>. At most one product may be offered to a customer in each cluster.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2_4"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-193-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-2101" style="width: 100%; display: inline-block; min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.086em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-2102"><span class="mtable" id="MathJax-Span-2103" style="min-width: 17.086em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.086em;"><span style="display: inline-block; position: absolute; width: 12.92em; height: 0px; clip: rect(-1.664em, 1012.92em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -6.485em;"><span style="position: absolute; clip: rect(2.324em, 1012.92em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 12.92em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1012.92em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.485em;"><span class="mtd" id="MathJax-Span-2107"><span class="mrow" id="MathJax-Span-2108"><span class="munderover" id="MathJax-Span-2109"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-2110" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2111"><span class="mrow" id="MathJax-Span-2112"><span class="mi" id="MathJax-Span-2113" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-2114" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2115"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2116" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-2117"><span class="mrow" id="MathJax-Span-2118"><span class="mi" id="MathJax-Span-2119" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-2120" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2121" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2122"><span class="mrow" id="MathJax-Span-2123"><span class="mi" id="MathJax-Span-2124" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2125" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2126" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2127" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2128" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2129" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-2130" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-2131" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-2132" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-2133" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2134" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-2135" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2136" style="font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2137"><span class="mrow" id="MathJax-Span-2138"><span class="mi" id="MathJax-Span-2139" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2140" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2141" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-2142" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-2143" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-2_4"><span class="mrow" id="MathJax-Span-2105"><span class="mtext" id="MathJax-Span-2106" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2_4"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-193" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}</script><ul>
<li><strong>Budget</strong>. Enforce budget constraint.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3_3"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;I&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msup&gt;&lt;mi&gt;B&lt;/mi&gt;&lt;mo&gt;&amp;#x2032;&lt;/mo&gt;&lt;/msup&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-194-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-2144" style="width: 100%; display: inline-block; min-width: 16.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 16.134em;"><span style="position: absolute; clip: rect(2.324em, 1011.97em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-2145"><span class="mtable" id="MathJax-Span-2146" style="min-width: 16.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 16.134em;"><span style="display: inline-block; position: absolute; width: 11.967em; height: 0px; clip: rect(-1.664em, 1011.97em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -6.009em;"><span style="position: absolute; clip: rect(2.324em, 1011.97em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 11.967em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1011.97em, 5.658em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.009em;"><span class="mtd" id="MathJax-Span-2150"><span class="mrow" id="MathJax-Span-2151"><span class="munderover" id="MathJax-Span-2152"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-2153" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-2154"><span class="mrow" id="MathJax-Span-2155"><span class="mi" id="MathJax-Span-2156" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2157" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-2158" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-2159" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-2160" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.289em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2161"><span class="mrow" id="MathJax-Span-2162"><span class="mi" id="MathJax-Span-2163" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2164" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2165"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2166" style="font-size: 70.7%; font-family: MathJax_Math-italic;">I<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.42em;"><span class="texatom" id="MathJax-Span-2167"><span class="mrow" id="MathJax-Span-2168"><span class="mi" id="MathJax-Span-2169" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-2170" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.61em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.063em;"><span class="mo" id="MathJax-Span-2171" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1001.61em, 4.408em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-2172"><span class="mrow" id="MathJax-Span-2173"><span class="mi" id="MathJax-Span-2174" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-2175" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="msubsup" id="MathJax-Span-2176"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2177" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.283em; left: 0.539em;"><span class="texatom" id="MathJax-Span-2178"><span class="mrow" id="MathJax-Span-2179"><span class="mi" id="MathJax-Span-2180" style="font-size: 50%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-2181" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.789em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2182" style="font-family: MathJax_Math-italic;">c</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-2183"><span class="mrow" id="MathJax-Span-2184"><span class="mi" id="MathJax-Span-2185" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2186" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2187" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2188" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2189" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2190" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-2191" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2192" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-2193"><span class="mrow" id="MathJax-Span-2194"><span class="mi" id="MathJax-Span-2195" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-2196" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2197" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-2198" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-2199" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-2200" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msup" id="MathJax-Span-2201" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-2202" style="font-family: MathJax_Math-italic;">B</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.777em;"><span class="mo" id="MathJax-Span-2203" style="font-size: 70.7%; font-family: MathJax_Main;">′</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-3_3"><span class="mrow" id="MathJax-Span-2148"><span class="mtext" id="MathJax-Span-2149" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.075em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3_3"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><msup><mi>I</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><msup><mi>J</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msup></mrow></munder><msub><mi>c</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>≤</mo><msup><mi>B</mi><mo>′</mo></msup></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-194" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{3}
\end{equation}</script>


In [ ]:

### Operational model enforcing constraint for total budget available 

# Declare and initialize model
mob = gp.Model('OperationalB')

### Decision variables

xb = mob.addVars(ccp, vtype=GRB.BINARY, name="assign")



In [ ]:

# Product offers constraint

productOffersb = {}

for k in clusters:
    for j in products:
            productOffersb[k,j] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffersb_' + str(k) + ',' + str(j) )



In [ ]:

# limit on the number of offers to each customer in a cluster.

customerOffersb = {}

for k,i in ki:
    customerOffersb[k,i] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffersb_' + str(k) + ',' + str(i) )



In [ ]:

# budget constraint

# New budget
new_budget = budget + z.x

totBudget = mob.addConstr(xb.prod(customer_cost) <= new_budget, name='total_budget')



In [ ]:

### Objective function

# Maximize total profit

mob.setObjective(xb.prod(customer_profit), GRB.MAXIMIZE)



In [ ]:

# Verify model formulation

mob.write('operationalB.lp')

# Run optimization engine

mob.optimize()



In [ ]:

### Output Reports

# Optimal assignment of product offers to customers

total_customer_profitb = 0
total_customer_costb = 0

kvalueb = None
firstb = True
num_assignmentsb = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalueb:
        prevkb = kvalueb
        kvalueb = k
        if not firstb:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevkb} is {num_assignmentsb}")
            print("___________________________________________________")
            num_assignmentsb = 0
        if firstb:
            firstb = False
    if xb[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profitb = '${:,.2f}'.format(customer_profit[k,i,j])
        costb = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profitb} at a cost of {costb}")
        total_customer_profitb += customer_profit[k,i,j]*xb[k,i,j].x
        total_customer_costb += customer_cost[k,i,j]*xb[k,i,j].x
        num_assignmentsb += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalueb} is {num_assignmentsb}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROIb = round(100*total_customer_profitb/total_customer_costb,2)

money_customers_profitb = '${:,.2f}'.format(total_customer_profitb)
money_customers_costb = '${:,.2f}'.format(total_customer_costb)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profitb}.")
print(f"Optimal total customers cost is {money_customers_costb} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROIb}% with a minimum ROI of  {min_ROI}%.")




<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Analysis">¶</a></h2><p>Each customer got, at most, one product offer. Products p1 and p2 are offerred to at least two customers. The cost of assigning  products to customers is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;600&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-195-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-2204" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1001.97em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2205"><span class="mi" id="MathJax-Span-2206" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-2207" style="font-family: MathJax_Main;">600</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>600</mn></math></span></span><script id="MathJax-Element-195" type="math/tex">\$600</script>, which is equal to the total budget available. The total customers profit is <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;6&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-196-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-2208" style="width: 3.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1002.92em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2209"><span class="mi" id="MathJax-Span-2210" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-2211" style="font-family: MathJax_Main;">6</span><span class="mo" id="MathJax-Span-2212" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-2213" style="font-family: MathJax_Main; padding-left: 0.182em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>6</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-196" type="math/tex">\$6,000</script>. The  ROI is 1,000 %, which is much higher than the minimum ROI required.</p>
<p>In this scenario, we enforce the total available budget constraint and we get a different distribution for the assignment. 
Product p1 is offered to customers c2 and c4, and product p2 is offered to customers c1 and c5.</p>



<h2 id="Conclusion">Conclusion<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#Conclusion">¶</a></h2><p>In this Jupyter Notebook, we discussed the importance of marketing campaigns for the banking industry. We discussed that machine learning predictive response models can be used to provide the input data of a marketing campaign optimization problem. We showed how the marketing campaign optimization problem can be decomposed into a tactical problem and an operational problem.</p>
<p>The tactical problem is formulated as a linear programming problem where we aggregate data generated by the machine learning predictive response models.</p>
<p>The solution of the tactical problem determines what products to offer to customers in clusters while maximizing the marketing campaign expected profits and considering the following constraints:</p>
<ul>
<li>limits on funding available for the campaign.</li>
<li>restrictions on the minimum number of product offers that can be made in a campaign.</li>
<li>campaign return-on-investment hurdle rates that must be met.</li>
</ul>
<p>The operational problem is formulated as a MIP model, where the expected customer  profits and the output of the tactical model can be used as inputs to assign products offers to individual customers in such way that the total customers profit is maximized. We considered two cases for the operational problem. In the first case, the total available budget determined by tactical problem is not enforced. This means that the optimal solution of this problem might have slight violations of the total available budget. In the second case, we enforce a total available budget.</p>



<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/marketing_campaign_optimization/marketing_campaign_optimization.html#References">¶</a></h2><p>[1] M. D. Cohen. <em>Exploiting response models—optimizing cross-sell and up-sell opportunities in banking.</em> Information Systems. Vol. 29. issue 4, June 2004, Pages 327-341</p>



<p>Copyright © 2020 Gurobi Optimization, LLC</p>
